In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
import calendar
import contextily as ctx
import glob
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBRegressor
import pickle

Initially I was planning to build a 2018's version of Cian's 2022 model, as use the prediction result as a baseline to compare with the event busyness. By such comparison, we get a relative busyness of event day vs non event days. Nevertheless, the outcome is not satisfied as the the difference is actually not significant as I thought ( primarily because of the inclusion of the event days themselves while training the models, as well as the less accuracy of model outcomes).

Alternatively, I decided to use the day of the week before the event day as baseline as Cian suggested. The details will be explained later in the following sections. As a result, the outcome is more promising at first glance, though it will have to wait frontend to visualised the comparison before we can take further action to improve our models.

As a result, the next couples of parts can be skipped as they are older work which does not reflect to the final work(or draft).

# Basic cleaning on 2018 taxi data (Skip this)

In [ ]:
# Using glob to load in all the parquet files
data_files = glob.glob('Summer Project/taxi/yellow_tripdata_2018-' + '*.parquet') 
data_files = sorted(data_files)

data_files

In [10]:
#read and get the taxi zone
taxi_zones = pd.read_csv("Summer Project/taxi_zones.csv")
taxi_zones

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
0,1,0.116357,MULTIPOLYGON (((-74.18445299999996 40.69499599...,0.000782,Newark Airport,1,EWR
1,2,0.433470,MULTIPOLYGON (((-73.82337597260663 40.63898704...,0.004866,Jamaica Bay,2,Queens
2,3,0.084341,MULTIPOLYGON (((-73.84792614099985 40.87134223...,0.000314,Allerton/Pelham Gardens,3,Bronx
3,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan
4,5,0.092146,MULTIPOLYGON (((-74.17421738099989 40.56256808...,0.000498,Arden Heights,5,Staten Island
...,...,...,...,...,...,...,...
258,256,0.067915,MULTIPOLYGON (((-73.95834207500002 40.71330630...,0.000169,Williamsburg (South Side),256,Brooklyn
259,259,0.126750,MULTIPOLYGON (((-73.85107116191898 40.91037152...,0.000395,Woodlawn/Wakefield,259,Bronx
260,260,0.133514,MULTIPOLYGON (((-73.90175373399988 40.76077547...,0.000422,Woodside,260,Queens
261,261,0.027120,MULTIPOLYGON (((-74.01332610899988 40.70503078...,0.000034,World Trade Center,261,Manhattan


In [11]:
manhattan_zones = taxi_zones[taxi_zones['borough'] == 'Manhattan']
manhattan_zones

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
3,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan
9,24,0.047000,MULTIPOLYGON (((-73.95953658899997 40.79871852...,0.000061,Bloomingdale,24,Manhattan
12,12,0.036661,MULTIPOLYGON (((-74.01565756599994 40.70483308...,0.000042,Battery Park,12,Manhattan
13,13,0.050281,MULTIPOLYGON (((-74.01244109299991 40.71905767...,0.000149,Battery Park City,13,Manhattan
40,41,0.052793,MULTIPOLYGON (((-73.94773985499985 40.80959972...,0.000143,Central Harlem,41,Manhattan
...,...,...,...,...,...,...,...
246,244,0.080569,MULTIPOLYGON (((-73.94068822000003 40.85131543...,0.000360,Washington Heights South,244,Manhattan
248,246,0.069467,MULTIPOLYGON (((-74.00439976203513 40.76267135...,0.000281,West Chelsea/Hudson Yards,246,Manhattan
251,249,0.036384,MULTIPOLYGON (((-74.00250642399995 40.72901638...,0.000072,West Village,249,Manhattan
261,261,0.027120,MULTIPOLYGON (((-74.01332610899988 40.70503078...,0.000034,World Trade Center,261,Manhattan


In [ ]:
# List to store each month in a dataframe
dataframes = []

#drop all the irrelevant features
columns_to_drop = ['VendorID', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
                  'PULocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
                  'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge',
                  'airport_fee']
for file in data_files:
    df = pd.read_parquet(file)
    
# normalise all the datetime features and create day of the week
    df['date'] = df['tpep_pickup_datetime'].dt.normalize()
    df['month'] = df['tpep_pickup_datetime'].dt.month
    df['time'] = df['tpep_pickup_datetime'].dt.hour
    df['day_of_the_week'] = df['date'].dt.day_name()
    df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

# drop all irrelevant features and null column
    df = df.drop(columns=columns_to_drop)
    df = df.dropna()
    df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
    
# Create an aggregated DataFrame
    df_agg = df.groupby(['DOLocationID', 'month', 'time', 'day_of_the_week']).size().reset_index(name='num_pickups')

# Join the aggregated DataFrame back to the original DataFrame
    df = pd.merge(df, df_agg, how='left', on=['DOLocationID', 'month', 'time', 'day_of_the_week'])
    
    df=df.sample(n=1000, random_state=1)
    
 # Append the month sample to the overall sample DataFrame   
    dataframes.append(df)

In [ ]:
# Concantenating all 12 months into one dataframe
df = pd.concat(dataframes)

In [ ]:
df = df.drop(columns="date")
df=df.drop("index", axis=1)

In [ ]:
mapping = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}

# Create an instance of LabelEncoder and fit the mapping
encoder = LabelEncoder()
encoder.fit([key for key, value in sorted(mapping.items(), key=lambda x: x[1])])

# Apply the custom mapping to label encode the column
df['day_of_the_week'] = df['day_of_the_week'].map(mapping)
df

In [ ]:
# save the ready-to-train data separately
df.to_csv('2018taxi.csv', index=False)

# 2018 whole year baseline(Skip this)

In [ ]:
df = pd.read_csv('2018taxi.csv')
df

In [ ]:
X = df.drop('num_pickups', axis=1)
y = df['num_pickups']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
X_test

In [ ]:
xg_reg = XGBRegressor(objective ='reg:squarederror', eval_metric ='rmse')
xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

In [ ]:
print("\n==================== Test Data =======================")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, preds))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, preds))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('R^2:', metrics.r2_score(y_test, preds))
print("\n=======================================================")

In [ ]:
# Define the parameter distribution for the randomized search
param_dist = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [2, 3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'min_child_weight': [1, 2, 3],
    'subsample': [0.5, 0.7, 1],
    'colsample_bytree': [0.5, 0.7, 1],
}

xg_reg = xgb.XGBRegressor(objective='reg:squarederror')

# Perform randomized search for best parameters
random_search = RandomizedSearchCV(estimator=xg_reg, param_distributions=param_dist, n_iter=100, cv=5, n_jobs=-1, verbose=1)
random_search.fit(X_train, y_train)

print(random_search.best_params_)

In [ ]:
# Applying the optimal hyperparameters
xg_reg = xgb.XGBRegressor(subsample=1, n_estimators=400, min_child_weight=2, max_depth=7, learning_rate=0.2, colsample_bytree=1, objective='reg:squarederror')

# Fit the regressor to the training set
xg_reg.fit(X_train, y_train)

# Predict on the test set
preds = xg_reg.predict(X_test)

In [ ]:
print("\n==================== Test Data =======================")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, preds))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, preds))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('R^2:', metrics.r2_score(y_test, preds))
print("\n=======================================================")

In [ ]:
data = pd.DataFrame({'DOLocationID': [239],
                     'month': [11],
                     'time': [8],
                     'day_of_the_week': [1]})


prediction = xg_reg.predict(data)
prediction

In [ ]:
with open('taxi2018.pkl', 'wb') as file:
    pickle.dump(xg_reg, file)

# Event data ML

This is the section where the final result are derived from. Details will be explained for the first event below (Times Square Ball). Since the same logic applied to all 8 events so understanding 1 event is enough to know how the whole things run.

# Times Square Ball

## actual data

Since this event happen in December, the taxi data for December is read. The datetime features of the data read is then normalised and day fo the week is calculated. Irrelvant features are dropped. Then, since the event happens on 2018-12-31, only data of the day is extracted.

In [13]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-12.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-12-31")]
df

,DOLocationID,date,month,time,day_of_the_week
4098378,234,2018-12-31,12,16,Monday
7956719,75,2018-12-31,12,0,Monday
7957133,88,2018-12-31,12,0,Monday
7957319,229,2018-12-31,12,0,Monday
7957320,141,2018-12-31,12,0,Monday
...,...,...,...,...,...
8195636,231,2018-12-31,12,22,Monday
8195657,100,2018-12-31,12,22,Monday
8195664,244,2018-12-31,12,23,Monday
8195669,107,2018-12-31,12,23,Monday


The next few steps group the data based on zoneID and time, and separate them into different items in a dict based on zone ID. The reason is because some zoneID does not have all 24 hour data(for eg 10am data might be absent), therefore we need to add those absent data in and assign 0 to the target features ("num_pickups"). Then, combine the result back into a df and it is ready to use. This will be the actual busyness of Times Square Ball at 2018-12-31.

In [14]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')
df_agg

,DOLocationID,time,num_pickups
0,4,0,47
1,4,1,37
2,4,2,20
3,4,3,15
4,4,4,13
...,...,...,...
1519,263,19,434
1520,263,20,358
1521,263,21,393
1522,263,22,278


In [15]:
separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0           47
1              4     1           37
2              4     2           20
3              4     3           15
4              4     4           13
5              4     5            6
6              4     6           13
7              4     7           12
8              4     8           14
9              4     9           17
10             4    10           33
11             4    11           34
12             4    12           37
13             4    13           46
14             4    14           58
15             4    15           66
16             4    16           99
17             4    17           92
18             4    18          133
19             4    19          135
20             4    20          110
21             4    21          109
22             4    22           91
23             4    23           76

Category: 12
    DOLocationID  time  num_pickups
24            12     0            2
25

In [16]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
        
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0           47
1              4     1           37
2              4     2           20
3              4     3           15
4              4     4           13
5              4     5            6
6              4     6           13
7              4     7           12
8              4     8           14
9              4     9           17
10             4    10           33
11             4    11           34
12             4    12           37
13             4    13           46
14             4    14           58
15             4    15           66
16             4    16           99
17             4    17           92
18             4    18          133
19             4    19          135
20             4    20          110
21             4    21          109
22             4    22           91
23             4    23           76

Category: 12
    DOLocationID  time  num_pickups
0             12     0            2
21

In [17]:
df_TimesSquare = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_TimesSquare= pd.concat([df_TimesSquare,df],axis=0)
# df_bl= pd.DataFrame(df_bl)
df_TimesSquare

,DOLocationID,time,num_pickups
0,4,0,47
1,4,1,37
2,4,2,20
3,4,3,15
4,4,4,13
...,...,...,...
1519,263,19,434
1520,263,20,358
1521,263,21,393
1522,263,22,278


## baseline

This section will get the data of the day a week before the event. Since the event happen in 2018-12-31, the data from 2018-12-25 will be rendered and cleaned. The process is similar as the previous section.

In [18]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-12.parquet")

In [19]:
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-12-25")]
df

,DOLocationID,date,month,time,day_of_the_week
4003682,162,2018-12-25,12,0,Tuesday
4003683,263,2018-12-25,12,0,Tuesday
6007597,162,2018-12-25,12,9,Tuesday
6007598,48,2018-12-25,12,9,Tuesday
6007599,263,2018-12-25,12,9,Tuesday
...,...,...,...,...,...
8191577,74,2018-12-25,12,21,Tuesday
8191609,261,2018-12-25,12,22,Tuesday
8191610,42,2018-12-25,12,22,Tuesday
8191613,243,2018-12-25,12,23,Tuesday


In [20]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

# Join the aggregated DataFrame back to the original DataFrame
# df = pd.merge(df["DOLocationID",'time'], df_agg, how='left', on=['DOLocationID', 'time'])
df_agg

,DOLocationID,time,num_pickups
0,4,0,41
1,4,1,34
2,4,2,22
3,4,3,15
4,4,4,7
...,...,...,...
1499,263,19,200
1500,263,20,183
1501,263,21,189
1502,263,22,161


In [21]:
separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0           41
1              4     1           34
2              4     2           22
3              4     3           15
4              4     4            7
5              4     5            3
6              4     6            4
7              4     7           10
8              4     8           13
9              4     9           15
10             4    10           16
11             4    11           15
12             4    12           19
13             4    13           17
14             4    14           33
15             4    15           24
16             4    16           27
17             4    17           38
18             4    18           53
19             4    19           51
20             4    20           47
21             4    21           55
22             4    22           52
23             4    23           38

Category: 12
    DOLocationID  time  num_pickups
24            12     2            1
25

In [22]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
        

In [23]:
dfs

{4:     DOLocationID  time  num_pickups
 0              4     0           41
 1              4     1           34
 2              4     2           22
 3              4     3           15
 4              4     4            7
 5              4     5            3
 6              4     6            4
 7              4     7           10
 8              4     8           13
 9              4     9           15
 10             4    10           16
 11             4    11           15
 12             4    12           19
 13             4    13           17
 14             4    14           33
 15             4    15           24
 16             4    16           27
 17             4    17           38
 18             4    18           53
 19             4    19           51
 20             4    20           47
 21             4    21           55
 22             4    22           52
 23             4    23           38,
 12:     DOLocationID  time  num_pickups
 16            12     0       

In [24]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
# df_bl= pd.DataFrame(df_bl)
df_bl

,DOLocationID,time,num_pickups
0,4,0,41
1,4,1,34
2,4,2,22
3,4,3,15
4,4,4,7
...,...,...,...
1499,263,19,200
1500,263,20,183
1501,263,21,189
1502,263,22,161


In [29]:
df_bl.reset_index(inplace=True)

In [30]:
df_bl=df_bl.drop("index",axis=1)
df_bl

,DOLocationID,time,num_pickups
0,4,0,41
1,4,1,34
2,4,2,22
3,4,3,15
4,4,4,7
...,...,...,...
1579,263,19,200
1580,263,20,183
1581,263,21,189
1582,263,22,161


In [31]:
df_TimesSquare.reset_index(drop=True, inplace=True)
df_TimesSquare = df_TimesSquare.rename(columns={'num_pickups':'num_pickups_TS'})
df_TimesSquare

,DOLocationID,time,num_pickups_TS
0,4,0,47
1,4,1,37
2,4,2,20
3,4,3,15
4,4,4,13
...,...,...,...
1579,263,19,434
1580,263,20,358
1581,263,21,393
1582,263,22,278


now we have actual busyness of 2018-12-31 and 2018-12-25. The next step is machine learning. We merged two data together and compare the "num_pickups" (busyness of 25th) and "num_pickups_TS" (busyness of 31th). From the result below, it is quite obvious that "num_pickups_TS" is generally higher than "num_pickups".

In [32]:
df_compare= pd.concat([df_bl,df_TimesSquare["num_pickups_TS"]],axis=1)
df_compare

,DOLocationID,time,num_pickups,num_pickups_TS
0,4,0,41,47
1,4,1,34,37
2,4,2,22,20
3,4,3,15,15
4,4,4,7,13
...,...,...,...,...
1579,263,19,200,434
1580,263,20,183,358
1581,263,21,189,393
1582,263,22,161,278


In [33]:
# this is just to show a rough comparison between average dropoff amount of 31st and 25th.
average = df_compare['num_pickups'].mean()
average_TS = df_compare['num_pickups_TS'].mean()
print(average)
print(average_TS)

67.75820707070707
116.13825757575758


## models and predictions

Split the data into 70:30 and use xgboost to train the model. 

In [34]:
X = df_compare.drop('num_pickups_TS', axis=1)
y = df_compare['num_pickups_TS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [35]:
xg_reg = XGBRegressor(objective ='reg:squarederror', eval_metric ='rmse')
xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

In [36]:
print("\n==================== Test Data =======================")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, preds))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, preds))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('R^2:', metrics.r2_score(y_test, preds))
print("\n=======================================================")


==================== Test Data =======================
Mean Absolute Error: 21.988568475767345
Mean Squared Error: 1152.9256657734468
Root Mean Squared Error: 33.954759103451856
R^2: 0.9377445121771744



In [37]:
# merge the prediction with the actual result to compare the difference
preds = xg_reg.predict(X_test)
true_vs_predicted_test = pd.DataFrame({'Actual Value': y_test, 'Predicted Value': preds})
true_vs_predicted_test

,Actual Value,Predicted Value
944,194,206.561935
1032,166,63.772053
893,70,73.737465
667,218,228.806671
1256,21,26.857512
...,...,...
1365,427,410.919403
842,0,1.395649
1199,188,126.310776
790,204,121.747025


So the prediction seems to be legit enough to use as the first version. Now we import the prediction from Cian's model, and search the corresponding result that matches the day of the event ( "month == 12" means December, "day_of_the_week== 5" means Saturday which are both derived from the actual day of event. Then same "DOLocationID" and "time" will be used. By doing so, we get a list of baseline busyness on December Saturday from Cian's Model.

In [38]:
df_2022 = pd.read_json('predictions.json')
df_2022 = df_2022[(df_2022['month'] == 12) & (df_2022['day_of_the_week'] == 5)]
df_2022.reset_index(drop=True, inplace=True)
df_2022=df_2022.drop(["month","day_of_the_week","busyness_score"], axis =1)
df_2022=df_2022.rename(columns={'ZoneID':'DOLocationID','predicted_dropoffs':'num_pickups'})
df_2022

,DOLocationID,time,num_pickups
0,236,17,1209.646240
1,236,21,956.360901
2,236,11,1263.923706
3,236,22,806.608948
4,236,9,827.831604
...,...,...,...
1603,105,23,621.909607
1604,105,16,537.766418
1605,105,8,242.729492
1606,105,3,306.604492


Now we feed all 4 features into our event model and get a prediction. The result is shown below.

In [39]:
preds = xg_reg.predict(df_2022)
true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
true_vs_predicted_test = pd.concat([df_2022,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
0,236,17,1209.646240,425.138733
1,236,21,956.360901,433.170471
2,236,11,1263.923706,466.482727
3,236,22,806.608948,400.927521
4,236,9,827.831604,464.487549
...,...,...,...,...
1603,105,23,621.909607,401.726929
1604,105,16,537.766418,429.524445
1605,105,8,242.729492,350.848419
1606,105,3,306.604492,359.698242


In [40]:
# sort the result by zoneID and time
true_vs_predicted_test = true_vs_predicted_test.sort_values(by=['DOLocationID', 'time'])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
610,4,0,150.663483,162.460297
618,4,1,151.560425,160.462357
615,4,2,152.729340,161.713699
622,4,3,105.289696,87.225334
612,4,4,24.593180,20.727751
...,...,...,...,...
369,263,19,670.643921,380.945099
374,263,20,724.976868,380.416077
361,263,21,738.318665,398.695435
363,263,22,762.412598,403.334747


In [42]:
# Normalize predictions to [0,1] range
scaler = MinMaxScaler()
true_vs_predicted_test['busyness_score'] = scaler.fit_transform(true_vs_predicted_test[['Predicted Value']])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score
610,4,0,150.663483,162.460297,0.268923
618,4,1,151.560425,160.462357,0.265701
615,4,2,152.729340,161.713699,0.267719
622,4,3,105.289696,87.225334,0.147587
612,4,4,24.593180,20.727751,0.040343
...,...,...,...,...,...
369,263,19,670.643921,380.945099,0.621287
374,263,20,724.976868,380.416077,0.620434
361,263,21,738.318665,398.695435,0.649914
363,263,22,762.412598,403.334747,0.657396


In [44]:
#attach an eventID to the data. The eventID is based on Harry's json
true_vs_predicted_test["Event_ID"]=5
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score,Event_ID
610,4,0,150.663483,162.460297,0.268923,5
618,4,1,151.560425,160.462357,0.265701,5
615,4,2,152.729340,161.713699,0.267719,5
622,4,3,105.289696,87.225334,0.147587,5
612,4,4,24.593180,20.727751,0.040343,5
...,...,...,...,...,...,...
369,263,19,670.643921,380.945099,0.621287,5
374,263,20,724.976868,380.416077,0.620434,5
361,263,21,738.318665,398.695435,0.649914,5
363,263,22,762.412598,403.334747,0.657396,5


In [53]:
#rearrange the df
true_vs_predicted_test = true_vs_predicted_test.rename(columns={'Predicted Value':'predicted_value'})
true_vs_predicted_test=true_vs_predicted_test[["Event_ID","DOLocationID","time","num_pickups","predicted_value","busyness_score"]]
true_vs_predicted_test.reset_index(drop=True, inplace=True)
true_vs_predicted_test

,Event_ID,DOLocationID,time,num_pickups,predicted_value,busyness_score
0,5,4,0,150.663483,162.460297,0.268923
1,5,4,1,151.560425,160.462357,0.265701
2,5,4,2,152.729340,161.713699,0.267719
3,5,4,3,105.289696,87.225334,0.147587
4,5,4,4,24.593180,20.727751,0.040343
...,...,...,...,...,...,...
1603,5,263,19,670.643921,380.945099,0.621287
1604,5,263,20,724.976868,380.416077,0.620434
1605,5,263,21,738.318665,398.695435,0.649914
1606,5,263,22,762.412598,403.334747,0.657396


In [54]:
#save the data and models separately
true_vs_predicted_test.to_json('TimesSquareBall.json', orient='records')

with open('TimesSquareBall.pkl', 'wb') as file:
    pickle.dump(xg_reg, file)

So the historical event prediction for Times Square Ball is completed. Same processes were carried out for the rest of 7 events and are shown below.

# Thanksgiving Day Parade

## actual data

In [56]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-11.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-11-22")]
df

,DOLocationID,date,month,time,day_of_the_week
4147083,107,2018-11-22,11,10,Thursday
4147084,162,2018-11-22,11,10,Thursday
4147085,209,2018-11-22,11,10,Thursday
4147086,148,2018-11-22,11,11,Thursday
4147087,186,2018-11-22,11,11,Thursday
...,...,...,...,...,...
8152170,238,2018-11-22,11,22,Thursday
8152175,90,2018-11-22,11,22,Thursday
8152178,74,2018-11-22,11,22,Thursday
8152182,116,2018-11-22,11,22,Thursday


In [57]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')
df_agg

,DOLocationID,time,num_pickups
0,4,0,55
1,4,1,60
2,4,2,23
3,4,3,36
4,4,4,22
...,...,...,...
1518,263,19,204
1519,263,20,242
1520,263,21,248
1521,263,22,211


In [58]:
separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0           55
1              4     1           60
2              4     2           23
3              4     3           36
4              4     4           22
5              4     5            7
6              4     6            7
7              4     7            8
8              4     8           21
9              4     9           14
10             4    10           24
11             4    11           26
12             4    12           34
13             4    13           36
14             4    14           37
15             4    15           41
16             4    16           46
17             4    17           67
18             4    18           55
19             4    19           58
20             4    20           64
21             4    21           75
22             4    22           84
23             4    23           65

Category: 12
    DOLocationID  time  num_pickups
24            12     0            2
25

In [59]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
        
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0           55
1              4     1           60
2              4     2           23
3              4     3           36
4              4     4           22
5              4     5            7
6              4     6            7
7              4     7            8
8              4     8           21
9              4     9           14
10             4    10           24
11             4    11           26
12             4    12           34
13             4    13           36
14             4    14           37
15             4    15           41
16             4    16           46
17             4    17           67
18             4    18           55
19             4    19           58
20             4    20           64
21             4    21           75
22             4    22           84
23             4    23           65

Category: 12
    DOLocationID  time  num_pickups
0             12     0            2
1 

In [60]:
df_Thanksgiving = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_Thanksgiving= pd.concat([df_Thanksgiving,df],axis=0)
df_Thanksgiving

,DOLocationID,time,num_pickups
0,4,0,55
1,4,1,60
2,4,2,23
3,4,3,36
4,4,4,22
...,...,...,...
1518,263,19,204
1519,263,20,242
1520,263,21,248
1521,263,22,211


## baseline

In [64]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-11.parquet")

In [65]:
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-11-15")]
df

,DOLocationID,date,month,time,day_of_the_week
3758494,140,2018-11-15,11,10,Thursday
3758495,237,2018-11-15,11,10,Thursday
3758496,170,2018-11-15,11,10,Thursday
3758497,164,2018-11-15,11,11,Thursday
3758500,237,2018-11-15,11,14,Thursday
...,...,...,...,...,...
4591219,48,2018-11-15,11,22,Thursday
4591220,42,2018-11-15,11,22,Thursday
8150156,164,2018-11-15,11,15,Thursday
8150157,113,2018-11-15,11,20,Thursday


In [66]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

# Join the aggregated DataFrame back to the original DataFrame
# df = pd.merge(df["DOLocationID",'time'], df_agg, how='left', on=['DOLocationID', 'time'])
df_agg

,DOLocationID,time,num_pickups
0,4,0,55
1,4,1,35
2,4,2,24
3,4,3,18
4,4,4,10
...,...,...,...
1505,263,19,221
1506,263,20,244
1507,263,21,238
1508,263,22,252


In [67]:
separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0           55
1              4     1           35
2              4     2           24
3              4     3           18
4              4     4           10
5              4     5           10
6              4     6            7
7              4     7           26
8              4     8           32
9              4     9           20
10             4    10           32
11             4    11           39
12             4    12           42
13             4    13           35
14             4    14           58
15             4    15           39
16             4    16           54
17             4    17           54
18             4    18           78
19             4    19           90
20             4    20          106
21             4    21          118
22             4    22          102
23             4    23           93

Category: 12
    DOLocationID  time  num_pickups
24            12     0            6
25

In [68]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
        

In [69]:
dfs

{4:     DOLocationID  time  num_pickups
 0              4     0           55
 1              4     1           35
 2              4     2           24
 3              4     3           18
 4              4     4           10
 5              4     5           10
 6              4     6            7
 7              4     7           26
 8              4     8           32
 9              4     9           20
 10             4    10           32
 11             4    11           39
 12             4    12           42
 13             4    13           35
 14             4    14           58
 15             4    15           39
 16             4    16           54
 17             4    17           54
 18             4    18           78
 19             4    19           90
 20             4    20          106
 21             4    21          118
 22             4    22          102
 23             4    23           93,
 12:     DOLocationID  time  num_pickups
 0             12     0       

In [70]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
# df_bl= pd.DataFrame(df_bl)
df_bl

,DOLocationID,time,num_pickups
0,4,0,55
1,4,1,35
2,4,2,24
3,4,3,18
4,4,4,10
...,...,...,...
1505,263,19,221
1506,263,20,244
1507,263,21,238
1508,263,22,252


In [71]:
df_bl.reset_index(inplace=True)
df_bl

,index,DOLocationID,time,num_pickups
0,0,4,0,55
1,1,4,1,35
2,2,4,2,24
3,3,4,3,18
4,4,4,4,10
...,...,...,...,...
1579,1505,263,19,221
1580,1506,263,20,244
1581,1507,263,21,238
1582,1508,263,22,252


In [72]:
df_bl=df_bl.drop("index",axis=1)

In [73]:
df_Thanksgiving.reset_index(drop=True, inplace=True)
df_Thanksgiving = df_Thanksgiving.rename(columns={'num_pickups':'num_pickups_TG'})
df_Thanksgiving

,DOLocationID,time,num_pickups_TG
0,4,0,55
1,4,1,60
2,4,2,23
3,4,3,36
4,4,4,22
...,...,...,...
1579,263,19,204
1580,263,20,242
1581,263,21,248
1582,263,22,211


In [74]:
df_compare= pd.concat([df_bl,df_Thanksgiving["num_pickups_TG"]],axis=1)
df_compare

,DOLocationID,time,num_pickups,num_pickups_TG
0,4,0,55,55
1,4,1,35,60
2,4,2,24,23
3,4,3,18,36
4,4,4,10,22
...,...,...,...,...
1579,263,19,221,204
1580,263,20,244,242
1581,263,21,238,248
1582,263,22,252,211


In [76]:
average = df_compare['num_pickups'].mean()
average_TG = df_compare['num_pickups_TG'].mean()
print(average)
print(average_TG)

128.46906565656565
94.8655303030303


## models and predictions

In [77]:
X = df_compare.drop('num_pickups_TG', axis=1)
y = df_compare['num_pickups_TG']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [78]:
xg_reg = XGBRegressor(objective ='reg:squarederror', eval_metric ='rmse')
xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

In [79]:
print("\n==================== Test Data =======================")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, preds))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, preds))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('R^2:', metrics.r2_score(y_test, preds))
print("\n=======================================================")


==================== Test Data =======================
Mean Absolute Error: 20.022520778598967
Mean Squared Error: 989.3231809213836
Root Mean Squared Error: 31.45350824504929
R^2: 0.8983892783841055



In [80]:
preds = xg_reg.predict(X_test)
true_vs_predicted_test = pd.DataFrame({'Actual Value': y_test, 'Predicted Value': preds})
# true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
# true_vs_predicted_test = pd.concat([X_test,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,Actual Value,Predicted Value
944,105,64.972260
1032,258,162.505066
893,36,107.362617
667,145,108.506134
1256,20,13.390193
...,...,...
1365,263,287.423737
842,1,1.727607
1199,109,156.559509
790,99,124.837288


In [81]:
df_2022 = pd.read_json('predictions.json')
df_2022 = df_2022[(df_2022['month'] == 11) & (df_2022['day_of_the_week'] == 3)]
df_2022.reset_index(drop=True, inplace=True)
df_2022=df_2022.drop(["month","day_of_the_week","busyness_score"], axis =1)
df_2022=df_2022.rename(columns={'ZoneID':'DOLocationID','predicted_dropoffs':'num_pickups'})
df_2022

,DOLocationID,time,num_pickups
0,236,17,1433.458496
1,236,21,1158.847778
2,236,11,1196.275024
3,236,22,868.462158
4,236,9,878.660217
...,...,...,...
1603,105,23,438.906616
1604,105,16,383.908752
1605,105,8,218.437286
1606,105,3,-3.223665


In [82]:
preds = xg_reg.predict(df_2022)
true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
true_vs_predicted_test = pd.concat([df_2022,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
0,236,17,1433.458496,415.730103
1,236,21,1158.847778,396.337036
2,236,11,1196.275024,355.690216
3,236,22,868.462158,422.240723
4,236,9,878.660217,226.690796
...,...,...,...,...
1603,105,23,438.906616,178.077957
1604,105,16,383.908752,288.184296
1605,105,8,218.437286,50.232201
1606,105,3,-3.223665,8.938803


In [83]:
true_vs_predicted_test = true_vs_predicted_test.sort_values(by=['DOLocationID', 'time'])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
610,4,0,81.660812,62.381664
618,4,1,63.463417,57.994789
615,4,2,9.878105,15.643648
622,4,3,1.346140,8.320813
612,4,4,-16.223701,3.933091
...,...,...,...,...
369,263,19,677.708130,373.155396
374,263,20,686.117310,373.259888
361,263,21,682.026245,369.682892
363,263,22,676.847168,371.574829


In [84]:
# Normalize predictions to [0,1] range
scaler = MinMaxScaler()
true_vs_predicted_test['busyness_score'] = scaler.fit_transform(true_vs_predicted_test[['Predicted Value']])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score
610,4,0,81.660812,62.381664,0.136378
618,4,1,63.463417,57.994789,0.128508
615,4,2,9.878105,15.643648,0.052531
622,4,3,1.346140,8.320813,0.039394
612,4,4,-16.223701,3.933091,0.031522
...,...,...,...,...,...
369,263,19,677.708130,373.155396,0.693898
374,263,20,686.117310,373.259888,0.694086
361,263,21,682.026245,369.682892,0.687669
363,263,22,676.847168,371.574829,0.691063


In [85]:
true_vs_predicted_test["Event_ID"]=1
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score,Event_ID
610,4,0,81.660812,62.381664,0.136378,1
618,4,1,63.463417,57.994789,0.128508,1
615,4,2,9.878105,15.643648,0.052531,1
622,4,3,1.346140,8.320813,0.039394,1
612,4,4,-16.223701,3.933091,0.031522,1
...,...,...,...,...,...,...
369,263,19,677.708130,373.155396,0.693898,1
374,263,20,686.117310,373.259888,0.694086,1
361,263,21,682.026245,369.682892,0.687669,1
363,263,22,676.847168,371.574829,0.691063,1


In [87]:
true_vs_predicted_test = true_vs_predicted_test.rename(columns={'Predicted Value':'predicted_value'})
true_vs_predicted_test=true_vs_predicted_test[["Event_ID","DOLocationID","time","num_pickups","predicted_value","busyness_score"]]
true_vs_predicted_test.reset_index(drop=True, inplace=True)
true_vs_predicted_test

,Event_ID,DOLocationID,time,num_pickups,predicted_value,busyness_score
0,1,4,0,81.660812,62.381664,0.136378
1,1,4,1,63.463417,57.994789,0.128508
2,1,4,2,9.878105,15.643648,0.052531
3,1,4,3,1.346140,8.320813,0.039394
4,1,4,4,-16.223701,3.933091,0.031522
...,...,...,...,...,...,...
1603,1,263,19,677.708130,373.155396,0.693898
1604,1,263,20,686.117310,373.259888,0.694086
1605,1,263,21,682.026245,369.682892,0.687669
1606,1,263,22,676.847168,371.574829,0.691063


In [88]:
true_vs_predicted_test.to_json('Thanksgiving.json', orient='records')

In [89]:
with open('Thanksgiving.pkl', 'wb') as file:
    pickle.dump(xg_reg, file)

# Halloween Parade

## actual data

In [90]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-10.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-10-28")]
df

,DOLocationID,date,month,time,day_of_the_week
1963989,237,2018-10-28,10,16,Sunday
7725484,13,2018-10-28,10,0,Sunday
7726733,4,2018-10-28,10,0,Sunday
7726734,148,2018-10-28,10,0,Sunday
7727519,234,2018-10-28,10,0,Sunday
...,...,...,...,...,...
8833042,74,2018-10-28,10,19,Sunday
8833045,42,2018-10-28,10,20,Sunday
8833048,234,2018-10-28,10,20,Sunday
8833051,48,2018-10-28,10,20,Sunday


In [92]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          132
1              4     1          134
2              4     2          118
3              4     3          104
4              4     4           66
5              4     5           27
6              4     6           14
7              4     7           16
8              4     8           20
9              4     9           19
10             4    10           28
11             4    11           40
12             4    12           67
13             4    13           69
14             4    14           43
15             4    15           56
16             4    16           57
17             4    17           65
18             4    18           80
19             4    19           64
20             4    20           80
21             4    21           59
22             4    22           60
23             4    23           56

Category: 12
    DOLocationID  time  num_pickups
24            12     0            1
25

In [93]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
        
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          132
1              4     1          134
2              4     2          118
3              4     3          104
4              4     4           66
5              4     5           27
6              4     6           14
7              4     7           16
8              4     8           20
9              4     9           19
10             4    10           28
11             4    11           40
12             4    12           67
13             4    13           69
14             4    14           43
15             4    15           56
16             4    16           57
17             4    17           65
18             4    18           80
19             4    19           64
20             4    20           80
21             4    21           59
22             4    22           60
23             4    23           56

Category: 12
    DOLocationID  time  num_pickups
0             12     0            1
1 

In [94]:
df_Halloween = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_Halloween= pd.concat([df_Halloween,df],axis=0)
df_Halloween

,DOLocationID,time,num_pickups
0,4,0,132
1,4,1,134
2,4,2,118
3,4,3,104
4,4,4,66
...,...,...,...
1514,263,19,324
1515,263,20,328
1516,263,21,237
1517,263,22,180


## baseline

In [96]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-10.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-10-21")]
df

,DOLocationID,date,month,time,day_of_the_week
5681032,142,2018-10-21,10,0,Sunday
5681104,243,2018-10-21,10,0,Sunday
5681268,79,2018-10-21,10,0,Sunday
5681523,142,2018-10-21,10,0,Sunday
5681751,186,2018-10-21,10,0,Sunday
...,...,...,...,...,...
8830071,42,2018-10-21,10,20,Sunday
8830079,238,2018-10-21,10,21,Sunday
8830093,137,2018-10-21,10,22,Sunday
8830095,151,2018-10-21,10,22,Sunday


In [98]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          113
1              4     1          117
2              4     2          108
3              4     3           81
4              4     4           36
5              4     5           20
6              4     6           10
7              4     7           15
8              4     8           36
9              4     9           26
10             4    10           37
11             4    11           43
12             4    12           71
13             4    13           51
14             4    14           54
15             4    15           62
16             4    16           68
17             4    17           89
18             4    18           79
19             4    19           79
20             4    20           75
21             4    21           74
22             4    22           83
23             4    23           52

Category: 12
    DOLocationID  time  num_pickups
24            12     0            4
25

In [99]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
dfs        

In [101]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
# df_bl= pd.DataFrame(df_bl)
df_bl

,DOLocationID,time,num_pickups
0,4,0,113
1,4,1,117
2,4,2,108
3,4,3,81
4,4,4,36
...,...,...,...
1525,263,19,365
1526,263,20,303
1527,263,21,270
1528,263,22,219


In [103]:
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl

,DOLocationID,time,num_pickups
0,4,0,113
1,4,1,117
2,4,2,108
3,4,3,81
4,4,4,36
...,...,...,...
1579,263,19,365
1580,263,20,303
1581,263,21,270
1582,263,22,219


In [104]:
df_Halloween.reset_index(drop=True, inplace=True)
df_Halloween = df_Halloween.rename(columns={'num_pickups':'num_pickups_HA'})
df_Halloween

,DOLocationID,time,num_pickups_HA
0,4,0,132
1,4,1,134
2,4,2,118
3,4,3,104
4,4,4,66
...,...,...,...
1579,263,19,324
1580,263,20,328
1581,263,21,237
1582,263,22,180


In [105]:
df_compare= pd.concat([df_bl,df_Halloween["num_pickups_HA"]],axis=1)
df_compare

,DOLocationID,time,num_pickups,num_pickups_HA
0,4,0,113,132
1,4,1,117,134
2,4,2,108,118
3,4,3,81,104
4,4,4,36,66
...,...,...,...,...
1579,263,19,365,324
1580,263,20,303,328
1581,263,21,270,237
1582,263,22,219,180


In [106]:
average = df_compare['num_pickups'].mean()
average_TS = df_compare['num_pickups_HA'].mean()
print(average)
print(average_TS)

147.94949494949495
142.80176767676767


## models and predictions

In [107]:
X = df_compare.drop('num_pickups_HA', axis=1)
y = df_compare['num_pickups_HA']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [108]:
xg_reg = XGBRegressor(objective ='reg:squarederror', eval_metric ='rmse')
xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

In [109]:
print("\n==================== Test Data =======================")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, preds))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, preds))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('R^2:', metrics.r2_score(y_test, preds))
print("\n=======================================================")


==================== Test Data =======================
Mean Absolute Error: 17.167854968491675
Mean Squared Error: 671.8015191502182
Root Mean Squared Error: 25.91913422840775
R^2: 0.9669544183373077



In [110]:
preds = xg_reg.predict(X_test)
true_vs_predicted_test = pd.DataFrame({'Actual Value': y_test, 'Predicted Value': preds})
# true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
# true_vs_predicted_test = pd.concat([X_test,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,Actual Value,Predicted Value
944,168,136.194244
1032,388,368.844421
893,24,40.590569
667,214,195.418472
1256,25,19.787485
...,...,...
1365,282,221.389923
842,2,-0.126118
1199,103,112.560059
790,109,128.540054


In [111]:
df_2022 = pd.read_json('predictions.json')
df_2022 = df_2022[(df_2022['month'] == 10) & (df_2022['day_of_the_week'] == 6)]
df_2022.reset_index(drop=True, inplace=True)
df_2022=df_2022.drop(["month","day_of_the_week","busyness_score"], axis =1)
df_2022=df_2022.rename(columns={'ZoneID':'DOLocationID','predicted_dropoffs':'num_pickups'})
df_2022

,DOLocationID,time,num_pickups
0,236,17,1093.374146
1,236,21,740.199707
2,236,11,1108.475586
3,236,22,424.808624
4,236,9,810.350159
...,...,...,...
1603,105,23,355.881012
1604,105,16,435.447205
1605,105,8,248.637070
1606,105,3,315.629456


In [112]:
preds = xg_reg.predict(df_2022)
true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
true_vs_predicted_test = pd.concat([df_2022,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
0,236,17,1093.374146,678.217834
1,236,21,740.199707,683.918518
2,236,11,1108.475586,719.382141
3,236,22,424.808624,387.014893
4,236,9,810.350159,706.087769
...,...,...,...,...
1603,105,23,355.881012,307.601776
1604,105,16,435.447205,358.655945
1605,105,8,248.637070,239.867264
1606,105,3,315.629456,341.087158


In [113]:
true_vs_predicted_test = true_vs_predicted_test.sort_values(by=['DOLocationID', 'time'])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
610,4,0,291.953583,332.935791
618,4,1,322.751312,342.175079
615,4,2,169.881012,212.704468
622,4,3,94.703430,88.119263
612,4,4,16.471519,24.451954
...,...,...,...,...
369,263,19,629.296997,578.280945
374,263,20,570.617493,494.536560
361,263,21,462.816803,365.900177
363,263,22,359.080841,283.281616


In [114]:
# Normalize predictions to [0,1] range
scaler = MinMaxScaler()
true_vs_predicted_test['busyness_score'] = scaler.fit_transform(true_vs_predicted_test[['Predicted Value']])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score
610,4,0,291.953583,332.935791,0.433820
618,4,1,322.751312,342.175079,0.445621
615,4,2,169.881012,212.704468,0.280258
622,4,3,94.703430,88.119263,0.121135
612,4,4,16.471519,24.451954,0.039817
...,...,...,...,...,...
369,263,19,629.296997,578.280945,0.747181
374,263,20,570.617493,494.536560,0.640221
361,263,21,462.816803,365.900177,0.475923
363,263,22,359.080841,283.281616,0.370401


In [115]:
true_vs_predicted_test["Event_ID"]=
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score,Event_ID
610,4,0,291.953583,332.935791,0.433820,2
618,4,1,322.751312,342.175079,0.445621,2
615,4,2,169.881012,212.704468,0.280258,2
622,4,3,94.703430,88.119263,0.121135,2
612,4,4,16.471519,24.451954,0.039817,2
...,...,...,...,...,...,...
369,263,19,629.296997,578.280945,0.747181,2
374,263,20,570.617493,494.536560,0.640221,2
361,263,21,462.816803,365.900177,0.475923,2
363,263,22,359.080841,283.281616,0.370401,2


In [116]:
true_vs_predicted_test = true_vs_predicted_test.rename(columns={'Predicted Value':'predicted_value'})
true_vs_predicted_test=true_vs_predicted_test[["Event_ID","DOLocationID","time","num_pickups","predicted_value","busyness_score"]]
true_vs_predicted_test.reset_index(drop=True, inplace=True)
true_vs_predicted_test

,Event_ID,DOLocationID,time,num_pickups,predicted_value,busyness_score
0,2,4,0,291.953583,332.935791,0.433820
1,2,4,1,322.751312,342.175079,0.445621
2,2,4,2,169.881012,212.704468,0.280258
3,2,4,3,94.703430,88.119263,0.121135
4,2,4,4,16.471519,24.451954,0.039817
...,...,...,...,...,...,...
1603,2,263,19,629.296997,578.280945,0.747181
1604,2,263,20,570.617493,494.536560,0.640221
1605,2,263,21,462.816803,365.900177,0.475923
1606,2,263,22,359.080841,283.281616,0.370401


# Halloween Parade

## actual data

In [90]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-10.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-10-28")]
df

,DOLocationID,date,month,time,day_of_the_week
1963989,237,2018-10-28,10,16,Sunday
7725484,13,2018-10-28,10,0,Sunday
7726733,4,2018-10-28,10,0,Sunday
7726734,148,2018-10-28,10,0,Sunday
7727519,234,2018-10-28,10,0,Sunday
...,...,...,...,...,...
8833042,74,2018-10-28,10,19,Sunday
8833045,42,2018-10-28,10,20,Sunday
8833048,234,2018-10-28,10,20,Sunday
8833051,48,2018-10-28,10,20,Sunday


In [92]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          132
1              4     1          134
2              4     2          118
3              4     3          104
4              4     4           66
5              4     5           27
6              4     6           14
7              4     7           16
8              4     8           20
9              4     9           19
10             4    10           28
11             4    11           40
12             4    12           67
13             4    13           69
14             4    14           43
15             4    15           56
16             4    16           57
17             4    17           65
18             4    18           80
19             4    19           64
20             4    20           80
21             4    21           59
22             4    22           60
23             4    23           56

Category: 12
    DOLocationID  time  num_pickups
24            12     0            1
25

In [93]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
        
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          132
1              4     1          134
2              4     2          118
3              4     3          104
4              4     4           66
5              4     5           27
6              4     6           14
7              4     7           16
8              4     8           20
9              4     9           19
10             4    10           28
11             4    11           40
12             4    12           67
13             4    13           69
14             4    14           43
15             4    15           56
16             4    16           57
17             4    17           65
18             4    18           80
19             4    19           64
20             4    20           80
21             4    21           59
22             4    22           60
23             4    23           56

Category: 12
    DOLocationID  time  num_pickups
0             12     0            1
1 

In [94]:
df_Halloween = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_Halloween= pd.concat([df_Halloween,df],axis=0)
df_Halloween

,DOLocationID,time,num_pickups
0,4,0,132
1,4,1,134
2,4,2,118
3,4,3,104
4,4,4,66
...,...,...,...
1514,263,19,324
1515,263,20,328
1516,263,21,237
1517,263,22,180


## baseline

In [96]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-10.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-10-21")]
df

,DOLocationID,date,month,time,day_of_the_week
5681032,142,2018-10-21,10,0,Sunday
5681104,243,2018-10-21,10,0,Sunday
5681268,79,2018-10-21,10,0,Sunday
5681523,142,2018-10-21,10,0,Sunday
5681751,186,2018-10-21,10,0,Sunday
...,...,...,...,...,...
8830071,42,2018-10-21,10,20,Sunday
8830079,238,2018-10-21,10,21,Sunday
8830093,137,2018-10-21,10,22,Sunday
8830095,151,2018-10-21,10,22,Sunday


In [98]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          113
1              4     1          117
2              4     2          108
3              4     3           81
4              4     4           36
5              4     5           20
6              4     6           10
7              4     7           15
8              4     8           36
9              4     9           26
10             4    10           37
11             4    11           43
12             4    12           71
13             4    13           51
14             4    14           54
15             4    15           62
16             4    16           68
17             4    17           89
18             4    18           79
19             4    19           79
20             4    20           75
21             4    21           74
22             4    22           83
23             4    23           52

Category: 12
    DOLocationID  time  num_pickups
24            12     0            4
25

In [99]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
dfs        

In [101]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
# df_bl= pd.DataFrame(df_bl)
df_bl

,DOLocationID,time,num_pickups
0,4,0,113
1,4,1,117
2,4,2,108
3,4,3,81
4,4,4,36
...,...,...,...
1525,263,19,365
1526,263,20,303
1527,263,21,270
1528,263,22,219


In [103]:
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl

,DOLocationID,time,num_pickups
0,4,0,113
1,4,1,117
2,4,2,108
3,4,3,81
4,4,4,36
...,...,...,...
1579,263,19,365
1580,263,20,303
1581,263,21,270
1582,263,22,219


In [104]:
df_Halloween.reset_index(drop=True, inplace=True)
df_Halloween = df_Halloween.rename(columns={'num_pickups':'num_pickups_HA'})
df_Halloween

,DOLocationID,time,num_pickups_HA
0,4,0,132
1,4,1,134
2,4,2,118
3,4,3,104
4,4,4,66
...,...,...,...
1579,263,19,324
1580,263,20,328
1581,263,21,237
1582,263,22,180


In [105]:
df_compare= pd.concat([df_bl,df_Halloween["num_pickups_HA"]],axis=1)
df_compare

,DOLocationID,time,num_pickups,num_pickups_HA
0,4,0,113,132
1,4,1,117,134
2,4,2,108,118
3,4,3,81,104
4,4,4,36,66
...,...,...,...,...
1579,263,19,365,324
1580,263,20,303,328
1581,263,21,270,237
1582,263,22,219,180


In [106]:
average = df_compare['num_pickups'].mean()
average_TS = df_compare['num_pickups_HA'].mean()
print(average)
print(average_TS)

147.94949494949495
142.80176767676767


## models and predictions

In [107]:
X = df_compare.drop('num_pickups_HA', axis=1)
y = df_compare['num_pickups_HA']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [108]:
xg_reg = XGBRegressor(objective ='reg:squarederror', eval_metric ='rmse')
xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

In [109]:
print("\n==================== Test Data =======================")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, preds))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, preds))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('R^2:', metrics.r2_score(y_test, preds))
print("\n=======================================================")


==================== Test Data =======================
Mean Absolute Error: 17.167854968491675
Mean Squared Error: 671.8015191502182
Root Mean Squared Error: 25.91913422840775
R^2: 0.9669544183373077



In [110]:
preds = xg_reg.predict(X_test)
true_vs_predicted_test = pd.DataFrame({'Actual Value': y_test, 'Predicted Value': preds})
# true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
# true_vs_predicted_test = pd.concat([X_test,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,Actual Value,Predicted Value
944,168,136.194244
1032,388,368.844421
893,24,40.590569
667,214,195.418472
1256,25,19.787485
...,...,...
1365,282,221.389923
842,2,-0.126118
1199,103,112.560059
790,109,128.540054


In [111]:
df_2022 = pd.read_json('predictions.json')
df_2022 = df_2022[(df_2022['month'] == 10) & (df_2022['day_of_the_week'] == 6)]
df_2022.reset_index(drop=True, inplace=True)
df_2022=df_2022.drop(["month","day_of_the_week","busyness_score"], axis =1)
df_2022=df_2022.rename(columns={'ZoneID':'DOLocationID','predicted_dropoffs':'num_pickups'})
df_2022

,DOLocationID,time,num_pickups
0,236,17,1093.374146
1,236,21,740.199707
2,236,11,1108.475586
3,236,22,424.808624
4,236,9,810.350159
...,...,...,...
1603,105,23,355.881012
1604,105,16,435.447205
1605,105,8,248.637070
1606,105,3,315.629456


In [112]:
preds = xg_reg.predict(df_2022)
true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
true_vs_predicted_test = pd.concat([df_2022,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
0,236,17,1093.374146,678.217834
1,236,21,740.199707,683.918518
2,236,11,1108.475586,719.382141
3,236,22,424.808624,387.014893
4,236,9,810.350159,706.087769
...,...,...,...,...
1603,105,23,355.881012,307.601776
1604,105,16,435.447205,358.655945
1605,105,8,248.637070,239.867264
1606,105,3,315.629456,341.087158


In [113]:
true_vs_predicted_test = true_vs_predicted_test.sort_values(by=['DOLocationID', 'time'])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
610,4,0,291.953583,332.935791
618,4,1,322.751312,342.175079
615,4,2,169.881012,212.704468
622,4,3,94.703430,88.119263
612,4,4,16.471519,24.451954
...,...,...,...,...
369,263,19,629.296997,578.280945
374,263,20,570.617493,494.536560
361,263,21,462.816803,365.900177
363,263,22,359.080841,283.281616


In [114]:
# Normalize predictions to [0,1] range
scaler = MinMaxScaler()
true_vs_predicted_test['busyness_score'] = scaler.fit_transform(true_vs_predicted_test[['Predicted Value']])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score
610,4,0,291.953583,332.935791,0.433820
618,4,1,322.751312,342.175079,0.445621
615,4,2,169.881012,212.704468,0.280258
622,4,3,94.703430,88.119263,0.121135
612,4,4,16.471519,24.451954,0.039817
...,...,...,...,...,...
369,263,19,629.296997,578.280945,0.747181
374,263,20,570.617493,494.536560,0.640221
361,263,21,462.816803,365.900177,0.475923
363,263,22,359.080841,283.281616,0.370401


In [115]:
true_vs_predicted_test["Event_ID"]=2
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score,Event_ID
610,4,0,291.953583,332.935791,0.433820,2
618,4,1,322.751312,342.175079,0.445621,2
615,4,2,169.881012,212.704468,0.280258,2
622,4,3,94.703430,88.119263,0.121135,2
612,4,4,16.471519,24.451954,0.039817,2
...,...,...,...,...,...,...
369,263,19,629.296997,578.280945,0.747181,2
374,263,20,570.617493,494.536560,0.640221,2
361,263,21,462.816803,365.900177,0.475923,2
363,263,22,359.080841,283.281616,0.370401,2


In [116]:
true_vs_predicted_test = true_vs_predicted_test.rename(columns={'Predicted Value':'predicted_value'})
true_vs_predicted_test=true_vs_predicted_test[["Event_ID","DOLocationID","time","num_pickups","predicted_value","busyness_score"]]
true_vs_predicted_test.reset_index(drop=True, inplace=True)
true_vs_predicted_test

,Event_ID,DOLocationID,time,num_pickups,predicted_value,busyness_score
0,2,4,0,291.953583,332.935791,0.433820
1,2,4,1,322.751312,342.175079,0.445621
2,2,4,2,169.881012,212.704468,0.280258
3,2,4,3,94.703430,88.119263,0.121135
4,2,4,4,16.471519,24.451954,0.039817
...,...,...,...,...,...,...
1603,2,263,19,629.296997,578.280945,0.747181
1604,2,263,20,570.617493,494.536560,0.640221
1605,2,263,21,462.816803,365.900177,0.475923
1606,2,263,22,359.080841,283.281616,0.370401


In [118]:
true_vs_predicted_test.to_json('Halloween.json', orient='records')
with open('Halloween.pkl', 'wb') as file:
    pickle.dump(xg_reg, file)

# Saint Patrick's Parade

## actual data

In [119]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-03.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-03-17")]
df

,DOLocationID,date,month,time,day_of_the_week
5013309,140,2018-03-17,3,0,Saturday
5014092,249,2018-03-17,3,0,Saturday
5014093,244,2018-03-17,3,0,Saturday
5014108,151,2018-03-17,3,0,Saturday
5015217,151,2018-03-17,3,0,Saturday
...,...,...,...,...,...
5611337,41,2018-03-17,3,18,Saturday
5622584,68,2018-03-17,3,18,Saturday
5630765,234,2018-03-17,3,23,Saturday
5634406,144,2018-03-17,3,23,Saturday


In [120]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          131
1              4     1          131
2              4     2           86
3              4     3           78
4              4     4           73
5              4     5           16
6              4     6           15
7              4     7           32
8              4     8           23
9              4     9           29
10             4    10           42
11             4    11           53
12             4    12           72
13             4    13           86
14             4    14           65
15             4    15           98
16             4    16           95
17             4    17           82
18             4    18          117
19             4    19          145
20             4    20          143
21             4    21          139
22             4    22          125
23             4    23          115

Category: 12
    DOLocationID  time  num_pickups
24            12     0            6
25

In [121]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
        
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          131
1              4     1          131
2              4     2           86
3              4     3           78
4              4     4           73
5              4     5           16
6              4     6           15
7              4     7           32
8              4     8           23
9              4     9           29
10             4    10           42
11             4    11           53
12             4    12           72
13             4    13           86
14             4    14           65
15             4    15           98
16             4    16           95
17             4    17           82
18             4    18          117
19             4    19          145
20             4    20          143
21             4    21          139
22             4    22          125
23             4    23          115

Category: 12
    DOLocationID  time  num_pickups
0             12     0            6
1 

In [122]:
df_SaintPatricks = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_SaintPatricks= pd.concat([df_SaintPatricks,df],axis=0)
df_SaintPatricks

,DOLocationID,time,num_pickups
0,4,0,131
1,4,1,131
2,4,2,86
3,4,3,78
4,4,4,73
...,...,...,...
1524,263,19,515
1525,263,20,461
1526,263,21,459
1527,263,22,465


## baseline

In [123]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-03.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-03-10")]
df

,DOLocationID,date,month,time,day_of_the_week
2758320,140,2018-03-10,3,0,Saturday
2759206,143,2018-03-10,3,23,Saturday
2759331,263,2018-03-10,3,0,Saturday
2759350,262,2018-03-10,3,0,Saturday
2759892,229,2018-03-10,3,0,Saturday
...,...,...,...,...,...
3372164,48,2018-03-10,3,18,Saturday
3383591,43,2018-03-10,3,18,Saturday
3383592,233,2018-03-10,3,18,Saturday
3395503,234,2018-03-10,3,23,Saturday


In [124]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          152
1              4     1          103
2              4     2          114
3              4     3           96
4              4     4           60
5              4     5           21
6              4     6           18
7              4     7           24
8              4     8           25
9              4     9           36
10             4    10           37
11             4    11           49
12             4    12           73
13             4    13           71
14             4    14           76
15             4    15          108
16             4    16           84
17             4    17           90
18             4    18          110
19             4    19          133
20             4    20          112
21             4    21          137
22             4    22           90
23             4    23          157

Category: 12
    DOLocationID  time  num_pickups
24            12     0            5
25

In [125]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
dfs        

{4:     DOLocationID  time  num_pickups
 0              4     0          152
 1              4     1          103
 2              4     2          114
 3              4     3           96
 4              4     4           60
 5              4     5           21
 6              4     6           18
 7              4     7           24
 8              4     8           25
 9              4     9           36
 10             4    10           37
 11             4    11           49
 12             4    12           73
 13             4    13           71
 14             4    14           76
 15             4    15          108
 16             4    16           84
 17             4    17           90
 18             4    18          110
 19             4    19          133
 20             4    20          112
 21             4    21          137
 22             4    22           90
 23             4    23          157,
 12:     DOLocationID  time  num_pickups
 0             12     0       

In [126]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
# df_bl= pd.DataFrame(df_bl)
df_bl

,DOLocationID,time,num_pickups
0,4,0,152
1,4,1,103
2,4,2,114
3,4,3,96
4,4,4,60
...,...,...,...
1521,263,19,413
1522,263,20,341
1523,263,21,353
1524,263,22,381


In [127]:
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl

,DOLocationID,time,num_pickups
0,4,0,152
1,4,1,103
2,4,2,114
3,4,3,96
4,4,4,60
...,...,...,...
1579,263,19,413
1580,263,20,341
1581,263,21,353
1582,263,22,381


In [128]:
df_SaintPatricks.reset_index(drop=True, inplace=True)
df_SaintPatricks = df_SaintPatricks.rename(columns={'num_pickups':'num_pickups_SP'})
df_SaintPatricks

,DOLocationID,time,num_pickups_SP
0,4,0,131
1,4,1,131
2,4,2,86
3,4,3,78
4,4,4,73
...,...,...,...
1579,263,19,515
1580,263,20,461
1581,263,21,459
1582,263,22,465


In [129]:
df_compare= pd.concat([df_bl,df_SaintPatricks["num_pickups_SP"]],axis=1)
df_compare

,DOLocationID,time,num_pickups,num_pickups_SP
0,4,0,152,131
1,4,1,103,131
2,4,2,114,86
3,4,3,96,78
4,4,4,60,73
...,...,...,...,...
1579,263,19,413,515
1580,263,20,341,461
1581,263,21,353,459
1582,263,22,381,465


In [130]:
average = df_compare['num_pickups'].mean()
average_TS = df_compare['num_pickups_SP'].mean()
print(average)
print(average_TS)

191.2840909090909
179.48358585858585


## models and predictions

In [131]:
X = df_compare.drop('num_pickups_SP', axis=1)
y = df_compare['num_pickups_SP']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [132]:
xg_reg = XGBRegressor(objective ='reg:squarederror', eval_metric ='rmse')
xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

In [133]:
print("\n==================== Test Data =======================")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, preds))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, preds))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('R^2:', metrics.r2_score(y_test, preds))
print("\n=======================================================")


==================== Test Data =======================
Mean Absolute Error: 20.983748802398683
Mean Squared Error: 1130.9243863653949
Root Mean Squared Error: 33.629219235144234
R^2: 0.9633649964393055



In [134]:
preds = xg_reg.predict(X_test)
true_vs_predicted_test = pd.DataFrame({'Actual Value': y_test, 'Predicted Value': preds})
# true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
# true_vs_predicted_test = pd.concat([X_test,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,Actual Value,Predicted Value
944,213,232.782837
1032,370,383.121460
893,62,45.083401
667,307,248.197891
1256,23,42.134590
...,...,...
1365,355,401.375854
842,0,2.726210
1199,396,387.674683
790,366,344.310120


In [135]:
df_2022 = pd.read_json('predictions.json')
df_2022 = df_2022[(df_2022['month'] == 3) & (df_2022['day_of_the_week'] == 6)]
df_2022.reset_index(drop=True, inplace=True)
df_2022=df_2022.drop(["month","day_of_the_week","busyness_score"], axis =1)
df_2022=df_2022.rename(columns={'ZoneID':'DOLocationID','predicted_dropoffs':'num_pickups'})
df_2022

,DOLocationID,time,num_pickups
0,236,17,939.583557
1,236,21,502.377075
2,236,11,887.642212
3,236,22,317.253876
4,236,9,506.865387
...,...,...,...
1603,105,23,239.050598
1604,105,16,467.481140
1605,105,8,213.161301
1606,105,3,306.130951


In [136]:
preds = xg_reg.predict(df_2022)
true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
true_vs_predicted_test = pd.concat([df_2022,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
0,236,17,939.583557,624.266663
1,236,21,502.377075,452.804108
2,236,11,887.642212,629.334839
3,236,22,317.253876,305.932556
4,236,9,506.865387,415.205353
...,...,...,...,...
1603,105,23,239.050598,275.897217
1604,105,16,467.481140,509.524506
1605,105,8,213.161301,255.077652
1606,105,3,306.130951,237.871658


In [137]:
true_vs_predicted_test = true_vs_predicted_test.sort_values(by=['DOLocationID', 'time'])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
610,4,0,166.349670,129.621384
618,4,1,150.785080,131.770142
615,4,2,130.828308,105.451035
622,4,3,79.112350,69.782715
612,4,4,42.189915,37.595070
...,...,...,...,...
369,263,19,554.083008,556.532227
374,263,20,444.670197,530.280151
361,263,21,411.711273,482.667114
363,263,22,340.783936,474.563110


In [138]:
# Normalize predictions to [0,1] range
scaler = MinMaxScaler()
true_vs_predicted_test['busyness_score'] = scaler.fit_transform(true_vs_predicted_test[['Predicted Value']])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score
610,4,0,166.349670,129.621384,0.148253
618,4,1,150.785080,131.770142,0.150537
615,4,2,130.828308,105.451035,0.122566
622,4,3,79.112350,69.782715,0.084660
612,4,4,42.189915,37.595070,0.050452
...,...,...,...,...,...
369,263,19,554.083008,556.532227,0.601952
374,263,20,444.670197,530.280151,0.574053
361,263,21,411.711273,482.667114,0.523452
363,263,22,340.783936,474.563110,0.514840


In [139]:
true_vs_predicted_test["Event_ID"]=3
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score,Event_ID
610,4,0,166.349670,129.621384,0.148253,3
618,4,1,150.785080,131.770142,0.150537,3
615,4,2,130.828308,105.451035,0.122566,3
622,4,3,79.112350,69.782715,0.084660,3
612,4,4,42.189915,37.595070,0.050452,3
...,...,...,...,...,...,...
369,263,19,554.083008,556.532227,0.601952,3
374,263,20,444.670197,530.280151,0.574053,3
361,263,21,411.711273,482.667114,0.523452,3
363,263,22,340.783936,474.563110,0.514840,3


In [140]:
true_vs_predicted_test = true_vs_predicted_test.rename(columns={'Predicted Value':'predicted_value'})
true_vs_predicted_test=true_vs_predicted_test[["Event_ID","DOLocationID","time","num_pickups","predicted_value","busyness_score"]]
true_vs_predicted_test.reset_index(drop=True, inplace=True)
true_vs_predicted_test

,Event_ID,DOLocationID,time,num_pickups,predicted_value,busyness_score
0,3,4,0,166.349670,129.621384,0.148253
1,3,4,1,150.785080,131.770142,0.150537
2,3,4,2,130.828308,105.451035,0.122566
3,3,4,3,79.112350,69.782715,0.084660
4,3,4,4,42.189915,37.595070,0.050452
...,...,...,...,...,...,...
1603,3,263,19,554.083008,556.532227,0.601952
1604,3,263,20,444.670197,530.280151,0.574053
1605,3,263,21,411.711273,482.667114,0.523452
1606,3,263,22,340.783936,474.563110,0.514840


In [141]:
true_vs_predicted_test.to_json('SaintPatricks.json', orient='records')
with open('SaintPatricks.pkl', 'wb') as file:
    pickle.dump(xg_reg, file)

# NYC Pride March

## actual data

In [146]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-06.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-06-24")]
df

,DOLocationID,date,month,time,day_of_the_week
6496829,239,2018-06-24,6,0,Sunday
6519484,107,2018-06-24,6,2,Sunday
6519485,140,2018-06-24,6,2,Sunday
6527202,161,2018-06-24,6,3,Sunday
6598864,90,2018-06-24,6,12,Sunday
...,...,...,...,...,...
7015439,68,2018-06-24,6,20,Sunday
7016321,41,2018-06-24,6,22,Sunday
7016322,236,2018-06-24,6,22,Sunday
7017463,244,2018-06-24,6,20,Sunday


In [147]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          115
1              4     1           95
2              4     2          107
3              4     3           73
4              4     4           56
5              4     5           49
6              4     6           19
7              4     7           25
8              4     8           28
9              4     9           20
10             4    10           35
11             4    11           56
12             4    12           62
13             4    13           61
14             4    14           73
15             4    15           58
16             4    16           69
17             4    17           90
18             4    18           79
19             4    19           89
20             4    20           76
21             4    21           97
22             4    22          105
23             4    23           81

Category: 12
    DOLocationID  time  num_pickups
24            12     0            4
25

In [148]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
        
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          115
1              4     1           95
2              4     2          107
3              4     3           73
4              4     4           56
5              4     5           49
6              4     6           19
7              4     7           25
8              4     8           28
9              4     9           20
10             4    10           35
11             4    11           56
12             4    12           62
13             4    13           61
14             4    14           73
15             4    15           58
16             4    16           69
17             4    17           90
18             4    18           79
19             4    19           89
20             4    20           76
21             4    21           97
22             4    22          105
23             4    23           81

Category: 12
    DOLocationID  time  num_pickups
0             12     0            4
1 

In [149]:
df_Pride = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_Pride= pd.concat([df_Pride,df],axis=0)
df_Pride

,DOLocationID,time,num_pickups
0,4,0,115
1,4,1,95
2,4,2,107
3,4,3,73
4,4,4,56
...,...,...,...
1526,263,19,330
1527,263,20,289
1528,263,21,271
1529,263,22,235


## baseline

In [150]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-06.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-06-17")]
df

,DOLocationID,date,month,time,day_of_the_week
3621651,68,2018-06-17,6,4,Sunday
4519004,141,2018-06-17,6,9,Sunday
4519005,170,2018-06-17,6,9,Sunday
4587318,74,2018-06-17,6,14,Sunday
4588152,161,2018-06-17,6,9,Sunday
...,...,...,...,...,...
4979378,237,2018-06-17,6,23,Sunday
4980365,230,2018-06-17,6,18,Sunday
4980366,161,2018-06-17,6,18,Sunday
4980368,114,2018-06-17,6,19,Sunday


In [151]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          112
1              4     1           98
2              4     2           72
3              4     3           52
4              4     4           38
5              4     5           13
6              4     6           11
7              4     7           18
8              4     8           19
9              4     9           43
10             4    10          100
11             4    11           57
12             4    12           51
13             4    13           71
14             4    14           69
15             4    15           66
16             4    16           61
17             4    17           59
18             4    18           98
19             4    19           73
20             4    20           70
21             4    21           99
22             4    22           67
23             4    23           54

Category: 12
    DOLocationID  time  num_pickups
24            12     0            8
25

In [152]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
dfs        

{4:     DOLocationID  time  num_pickups
 0              4     0          112
 1              4     1           98
 2              4     2           72
 3              4     3           52
 4              4     4           38
 5              4     5           13
 6              4     6           11
 7              4     7           18
 8              4     8           19
 9              4     9           43
 10             4    10          100
 11             4    11           57
 12             4    12           51
 13             4    13           71
 14             4    14           69
 15             4    15           66
 16             4    16           61
 17             4    17           59
 18             4    18           98
 19             4    19           73
 20             4    20           70
 21             4    21           99
 22             4    22           67
 23             4    23           54,
 12:     DOLocationID  time  num_pickups
 24            12     0       

In [153]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
# df_bl= pd.DataFrame(df_bl)
df_bl

,DOLocationID,time,num_pickups
0,4,0,112
1,4,1,98
2,4,2,72
3,4,3,52
4,4,4,38
...,...,...,...
1538,263,19,303
1539,263,20,282
1540,263,21,312
1541,263,22,252


In [154]:
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl

,DOLocationID,time,num_pickups
0,4,0,112
1,4,1,98
2,4,2,72
3,4,3,52
4,4,4,38
...,...,...,...
1579,263,19,303
1580,263,20,282
1581,263,21,312
1582,263,22,252


In [155]:
df_Pride.reset_index(drop=True, inplace=True)
df_Pride = df_Pride.rename(columns={'num_pickups':'num_pickups_PR'})
df_Pride

,DOLocationID,time,num_pickups_PR
0,4,0,115
1,4,1,95
2,4,2,107
3,4,3,73
4,4,4,56
...,...,...,...
1579,263,19,330
1580,263,20,289
1581,263,21,271
1582,263,22,235


In [156]:
df_compare= pd.concat([df_bl,df_Pride["num_pickups_PR"]],axis=1)
df_compare

,DOLocationID,time,num_pickups,num_pickups_PR
0,4,0,112,115
1,4,1,98,95
2,4,2,72,107
3,4,3,52,73
4,4,4,38,56
...,...,...,...,...
1579,263,19,303,330
1580,263,20,282,289
1581,263,21,312,271
1582,263,22,252,235


In [157]:
average = df_compare['num_pickups'].mean()
average_TS = df_compare['num_pickups_PR'].mean()
print(average)
print(average_TS)

133.06376262626262
131.57323232323233


## models and predictions

In [158]:
X = df_compare.drop('num_pickups_PR', axis=1)
y = df_compare['num_pickups_PR']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [159]:
xg_reg = XGBRegressor(objective ='reg:squarederror', eval_metric ='rmse')
xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

In [160]:
print("\n==================== Test Data =======================")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, preds))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, preds))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('R^2:', metrics.r2_score(y_test, preds))
print("\n=======================================================")


==================== Test Data =======================
Mean Absolute Error: 19.169457681132716
Mean Squared Error: 846.5712549038835
Root Mean Squared Error: 29.095897561406893
R^2: 0.9484874480279013



In [161]:
preds = xg_reg.predict(X_test)
true_vs_predicted_test = pd.DataFrame({'Actual Value': y_test, 'Predicted Value': preds})
# true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
# true_vs_predicted_test = pd.concat([X_test,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,Actual Value,Predicted Value
944,76,96.420181
1032,377,405.065826
893,43,42.609783
667,180,178.599686
1256,15,9.428840
...,...,...
1365,283,250.645233
842,0,2.101729
1199,165,142.409012
790,108,163.187592


In [162]:
df_2022 = pd.read_json('predictions.json')
df_2022 = df_2022[(df_2022['month'] == 6) & (df_2022['day_of_the_week'] == 6)]
df_2022.reset_index(drop=True, inplace=True)
df_2022=df_2022.drop(["month","day_of_the_week","busyness_score"], axis =1)
df_2022=df_2022.rename(columns={'ZoneID':'DOLocationID','predicted_dropoffs':'num_pickups'})
df_2022

,DOLocationID,time,num_pickups
0,236,17,794.106995
1,236,21,505.667236
2,236,11,825.072876
3,236,22,333.393219
4,236,9,478.547760
...,...,...,...
1603,105,23,283.534637
1604,105,16,410.832886
1605,105,8,223.506836
1606,105,3,253.072815


In [163]:
preds = xg_reg.predict(df_2022)
true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
true_vs_predicted_test = pd.concat([df_2022,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
0,236,17,794.106995,565.590149
1,236,21,505.667236,450.716797
2,236,11,825.072876,537.468750
3,236,22,333.393219,242.929047
4,236,9,478.547760,360.731689
...,...,...,...,...
1603,105,23,283.534637,267.493195
1604,105,16,410.832886,350.845917
1605,105,8,223.506836,326.897491
1606,105,3,253.072815,362.671692


In [164]:
true_vs_predicted_test = true_vs_predicted_test.sort_values(by=['DOLocationID', 'time'])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
610,4,0,119.263512,136.993240
618,4,1,135.615585,136.730606
615,4,2,123.787895,137.552643
622,4,3,36.568146,55.029526
612,4,4,-10.437315,5.485847
...,...,...,...,...
369,263,19,459.980743,412.304199
374,263,20,491.788727,398.685974
361,263,21,457.102142,396.557739
363,263,22,371.666748,328.575195


In [165]:
# Normalize predictions to [0,1] range
scaler = MinMaxScaler()
true_vs_predicted_test['busyness_score'] = scaler.fit_transform(true_vs_predicted_test[['Predicted Value']])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score
610,4,0,119.263512,136.993240,0.192002
618,4,1,135.615585,136.730606,0.191641
615,4,2,123.787895,137.552643,0.192770
622,4,3,36.568146,55.029526,0.079365
612,4,4,-10.437315,5.485847,0.011281
...,...,...,...,...,...
369,263,19,459.980743,412.304199,0.570342
374,263,20,491.788727,398.685974,0.551627
361,263,21,457.102142,396.557739,0.548703
363,263,22,371.666748,328.575195,0.455279


In [166]:
true_vs_predicted_test["Event_ID"]=4
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score,Event_ID
610,4,0,119.263512,136.993240,0.192002,4
618,4,1,135.615585,136.730606,0.191641,4
615,4,2,123.787895,137.552643,0.192770,4
622,4,3,36.568146,55.029526,0.079365,4
612,4,4,-10.437315,5.485847,0.011281,4
...,...,...,...,...,...,...
369,263,19,459.980743,412.304199,0.570342,4
374,263,20,491.788727,398.685974,0.551627,4
361,263,21,457.102142,396.557739,0.548703,4
363,263,22,371.666748,328.575195,0.455279,4


In [167]:
true_vs_predicted_test = true_vs_predicted_test.rename(columns={'Predicted Value':'predicted_value'})
true_vs_predicted_test=true_vs_predicted_test[["Event_ID","DOLocationID","time","num_pickups","predicted_value","busyness_score"]]
true_vs_predicted_test.reset_index(drop=True, inplace=True)
true_vs_predicted_test

,Event_ID,DOLocationID,time,num_pickups,predicted_value,busyness_score
0,4,4,0,119.263512,136.993240,0.192002
1,4,4,1,135.615585,136.730606,0.191641
2,4,4,2,123.787895,137.552643,0.192770
3,4,4,3,36.568146,55.029526,0.079365
4,4,4,4,-10.437315,5.485847,0.011281
...,...,...,...,...,...,...
1603,4,263,19,459.980743,412.304199,0.570342
1604,4,263,20,491.788727,398.685974,0.551627
1605,4,263,21,457.102142,396.557739,0.548703
1606,4,263,22,371.666748,328.575195,0.455279


In [168]:
true_vs_predicted_test.to_json('Pride.json', orient='records')
with open('Pride.pkl', 'wb') as file:
    pickle.dump(xg_reg, file)

# Lunar New Year Parade

## actual data

In [173]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-02.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-02-25")]
df

,DOLocationID,date,month,time,day_of_the_week
7307338,87,2018-02-25,2,0,Sunday
7310427,237,2018-02-25,2,0,Sunday
7310835,90,2018-02-25,2,0,Sunday
7310923,41,2018-02-25,2,0,Sunday
7310969,79,2018-02-25,2,0,Sunday
...,...,...,...,...,...
7601755,144,2018-02-25,2,23,Sunday
7601761,163,2018-02-25,2,23,Sunday
7602418,158,2018-02-25,2,20,Sunday
7603926,230,2018-02-25,2,23,Sunday


In [174]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          143
1              4     1          137
2              4     2          116
3              4     3          110
4              4     4           86
5              4     5           29
6              4     6           15
7              4     7           16
8              4     8           17
9              4     9           26
10             4    10           40
11             4    11           47
12             4    12           52
13             4    13           47
14             4    14           48
15             4    15           74
16             4    16           69
17             4    17           74
18             4    18           95
19             4    19           76
20             4    20           82
21             4    21           66
22             4    22           61
23             4    23           55

Category: 12
    DOLocationID  time  num_pickups
24            12     0            8
25

In [175]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
        
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          143
1              4     1          137
2              4     2          116
3              4     3          110
4              4     4           86
5              4     5           29
6              4     6           15
7              4     7           16
8              4     8           17
9              4     9           26
10             4    10           40
11             4    11           47
12             4    12           52
13             4    13           47
14             4    14           48
15             4    15           74
16             4    16           69
17             4    17           74
18             4    18           95
19             4    19           76
20             4    20           82
21             4    21           66
22             4    22           61
23             4    23           55

Category: 12
    DOLocationID  time  num_pickups
0             12     0            8
1 

     DOLocationID  time  num_pickups
821           158     0          293
822           158     1          184
823           158     2          157
824           158     3          103
825           158     4           41
826           158     5            7
827           158     6           12
828           158     7           17
829           158     8           51
830           158     9           69
831           158    10          117
832           158    11          169
833           158    12          183
834           158    13          189
835           158    14          170
836           158    15          168
837           158    16          154
838           158    17          171
839           158    18          183
840           158    19          140
841           158    20          106
842           158    21          120
843           158    22           90
844           158    23           56

Category: 161
     DOLocationID  time  num_pickups
845           161     0

In [176]:
df_LNY= pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_LNY= pd.concat([df_LNY,df],axis=0)
df_LNY

,DOLocationID,time,num_pickups
0,4,0,143
1,4,1,137
2,4,2,116
3,4,3,110
4,4,4,86
...,...,...,...
1523,263,19,331
1524,263,20,266
1525,263,21,173
1526,263,22,189


## baseline

In [177]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-02.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-02-18")]
df

,DOLocationID,date,month,time,day_of_the_week
5281360,142,2018-02-18,2,0,Sunday
5281726,13,2018-02-18,2,23,Sunday
5281739,151,2018-02-18,2,0,Sunday
5282277,158,2018-02-18,2,0,Sunday
5282405,158,2018-02-18,2,0,Sunday
...,...,...,...,...,...
5829043,158,2018-02-18,2,23,Sunday
5844790,43,2018-02-18,2,23,Sunday
5886733,236,2018-02-18,2,23,Sunday
5886734,237,2018-02-18,2,23,Sunday


In [178]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0          138
1              4     1          130
2              4     2          116
3              4     3           83
4              4     4           42
5              4     5           14
6              4     6           12
7              4     7           14
8              4     8           13
9              4     9           13
10             4    10           26
11             4    11           41
12             4    12           45
13             4    13           65
14             4    14           53
15             4    15           75
16             4    16           64
17             4    17           88
18             4    18           85
19             4    19           83
20             4    20           68
21             4    21           78
22             4    22           88
23             4    23          105

Category: 12
    DOLocationID  time  num_pickups
24            12     0            5
25

In [179]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
dfs        

{4:     DOLocationID  time  num_pickups
 0              4     0          138
 1              4     1          130
 2              4     2          116
 3              4     3           83
 4              4     4           42
 5              4     5           14
 6              4     6           12
 7              4     7           14
 8              4     8           13
 9              4     9           13
 10             4    10           26
 11             4    11           41
 12             4    12           45
 13             4    13           65
 14             4    14           53
 15             4    15           75
 16             4    16           64
 17             4    17           88
 18             4    18           85
 19             4    19           83
 20             4    20           68
 21             4    21           78
 22             4    22           88
 23             4    23          105,
 12:     DOLocationID  time  num_pickups
 0             12     0       

In [180]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
# df_bl= pd.DataFrame(df_bl)
df_bl

,DOLocationID,time,num_pickups
0,4,0,138
1,4,1,130
2,4,2,116
3,4,3,83
4,4,4,42
...,...,...,...
1522,263,19,281
1523,263,20,258
1524,263,21,268
1525,263,22,208


In [181]:
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl

,DOLocationID,time,num_pickups
0,4,0,138
1,4,1,130
2,4,2,116
3,4,3,83
4,4,4,42
...,...,...,...
1579,263,19,281
1580,263,20,258
1581,263,21,268
1582,263,22,208


In [182]:
df_LNY.reset_index(drop=True, inplace=True)
df_LNY = df_LNY.rename(columns={'num_pickups':'num_pickups_LNY'})
df_LNY

,DOLocationID,time,num_pickups_LNY
0,4,0,143
1,4,1,137
2,4,2,116
3,4,3,110
4,4,4,86
...,...,...,...
1579,263,19,331
1580,263,20,266
1581,263,21,173
1582,263,22,189


In [183]:
df_compare= pd.concat([df_bl,df_LNY["num_pickups_LNY"]],axis=1)
df_compare

,DOLocationID,time,num_pickups,num_pickups_LNY
0,4,0,138,143
1,4,1,130,137
2,4,2,116,116
3,4,3,83,110
4,4,4,42,86
...,...,...,...,...
1579,263,19,281,331
1580,263,20,258,266
1581,263,21,268,173
1582,263,22,208,189


In [184]:
average = df_compare['num_pickups'].mean()
average_TS = df_compare['num_pickups_LNY'].mean()
print(average)
print(average_TS)

139.51578282828282
148.2405303030303


## models and predictions

In [185]:
X = df_compare.drop('num_pickups_LNY', axis=1)
y = df_compare['num_pickups_LNY']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [186]:
xg_reg = XGBRegressor(objective ='reg:squarederror', eval_metric ='rmse')
xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

In [187]:
print("\n==================== Test Data =======================")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, preds))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, preds))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('R^2:', metrics.r2_score(y_test, preds))
print("\n=======================================================")


==================== Test Data =======================
Mean Absolute Error: 19.197968098358448
Mean Squared Error: 1029.0541615762745
Root Mean Squared Error: 32.07887406964706
R^2: 0.9543169572601915



In [188]:
preds = xg_reg.predict(X_test)
true_vs_predicted_test = pd.DataFrame({'Actual Value': y_test, 'Predicted Value': preds})
# true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
# true_vs_predicted_test = pd.concat([X_test,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,Actual Value,Predicted Value
944,133,125.643486
1032,421,457.689667
893,36,36.854229
667,203,182.859558
1256,27,73.310272
...,...,...
1365,212,202.778305
842,3,1.445585
1199,92,97.855415
790,105,133.684402


In [189]:
df_2022 = pd.read_json('predictions.json')
df_2022 = df_2022[(df_2022['month'] == 2) & (df_2022['day_of_the_week'] == 6)]
df_2022.reset_index(drop=True, inplace=True)
df_2022=df_2022.drop(["month","day_of_the_week","busyness_score"], axis =1)
df_2022=df_2022.rename(columns={'ZoneID':'DOLocationID','predicted_dropoffs':'num_pickups'})
df_2022

,DOLocationID,time,num_pickups
0,236,17,941.518127
1,236,21,450.592285
2,236,11,890.457581
3,236,22,345.102600
4,236,9,479.586182
...,...,...,...
1603,105,23,220.680145
1604,105,16,450.854797
1605,105,8,204.770111
1606,105,3,284.633545


In [190]:
preds = xg_reg.predict(df_2022)
true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
true_vs_predicted_test = pd.concat([df_2022,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
0,236,17,941.518127,618.954163
1,236,21,450.592285,507.301453
2,236,11,890.457581,772.916504
3,236,22,345.102600,346.620636
4,236,9,479.586182,733.631470
...,...,...,...,...
1603,105,23,220.680145,131.507782
1604,105,16,450.854797,423.197937
1605,105,8,204.770111,279.473022
1606,105,3,284.633545,429.656311


In [191]:
true_vs_predicted_test = true_vs_predicted_test.sort_values(by=['DOLocationID', 'time'])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
610,4,0,152.102829,132.741791
618,4,1,133.872452,133.172714
615,4,2,143.862564,121.758507
622,4,3,97.434273,108.774696
612,4,4,68.640625,93.639648
...,...,...,...,...
369,263,19,426.634064,350.282013
374,263,20,372.555969,280.978363
361,263,21,350.228455,255.790009
363,263,22,284.423309,260.832764


In [192]:
# Normalize predictions to [0,1] range
scaler = MinMaxScaler()
true_vs_predicted_test['busyness_score'] = scaler.fit_transform(true_vs_predicted_test[['Predicted Value']])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score
610,4,0,152.102829,132.741791,0.158352
618,4,1,133.872452,133.172714,0.158816
615,4,2,143.862564,121.758507,0.146525
622,4,3,97.434273,108.774696,0.132543
612,4,4,68.640625,93.639648,0.116244
...,...,...,...,...,...
369,263,19,426.634064,350.282013,0.392618
374,263,20,372.555969,280.978363,0.317986
361,263,21,350.228455,255.790009,0.290861
363,263,22,284.423309,260.832764,0.296291


In [193]:
true_vs_predicted_test["Event_ID"]=6
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score,Event_ID
610,4,0,152.102829,132.741791,0.158352,6
618,4,1,133.872452,133.172714,0.158816,6
615,4,2,143.862564,121.758507,0.146525,6
622,4,3,97.434273,108.774696,0.132543,6
612,4,4,68.640625,93.639648,0.116244,6
...,...,...,...,...,...,...
369,263,19,426.634064,350.282013,0.392618,6
374,263,20,372.555969,280.978363,0.317986,6
361,263,21,350.228455,255.790009,0.290861,6
363,263,22,284.423309,260.832764,0.296291,6


In [194]:
true_vs_predicted_test = true_vs_predicted_test.rename(columns={'Predicted Value':'predicted_value'})
true_vs_predicted_test=true_vs_predicted_test[["Event_ID","DOLocationID","time","num_pickups","predicted_value","busyness_score"]]
true_vs_predicted_test.reset_index(drop=True, inplace=True)
true_vs_predicted_test

,Event_ID,DOLocationID,time,num_pickups,predicted_value,busyness_score
0,6,4,0,152.102829,132.741791,0.158352
1,6,4,1,133.872452,133.172714,0.158816
2,6,4,2,143.862564,121.758507,0.146525
3,6,4,3,97.434273,108.774696,0.132543
4,6,4,4,68.640625,93.639648,0.116244
...,...,...,...,...,...,...
1603,6,263,19,426.634064,350.282013,0.392618
1604,6,263,20,372.555969,280.978363,0.317986
1605,6,263,21,350.228455,255.790009,0.290861
1606,6,263,22,284.423309,260.832764,0.296291


In [195]:
true_vs_predicted_test.to_json('LNY.json', orient='records')
with open('LNY.pkl', 'wb') as file:
    pickle.dump(xg_reg, file)

# Three Kings Day Parade

## actual data

In [196]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-01.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-01-05")]
df

,DOLocationID,date,month,time,day_of_the_week
859831,224,2018-01-05,1,0,Friday
859885,50,2018-01-05,1,0,Friday
860029,161,2018-01-05,1,0,Friday
860303,4,2018-01-05,1,0,Friday
860778,158,2018-01-05,1,0,Friday
...,...,...,...,...,...
1156426,107,2018-01-05,1,22,Friday
1159228,13,2018-01-05,1,23,Friday
1160958,161,2018-01-05,1,7,Friday
1160959,90,2018-01-05,1,7,Friday


In [197]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0           28
1              4     1           18
2              4     2           11
3              4     3           11
4              4     4           10
5              4     5            5
6              4     6           10
7              4     7           21
8              4     8           39
9              4     9           30
10             4    10           25
11             4    11           34
12             4    12           44
13             4    13           51
14             4    14           48
15             4    15           66
16             4    16           84
17             4    17          100
18             4    18          135
19             4    19          176
20             4    20          122
21             4    21          140
22             4    22          125
23             4    23          136

Category: 12
    DOLocationID  time  num_pickups
24            12     0            3
25

In [198]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
        
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0           28
1              4     1           18
2              4     2           11
3              4     3           11
4              4     4           10
5              4     5            5
6              4     6           10
7              4     7           21
8              4     8           39
9              4     9           30
10             4    10           25
11             4    11           34
12             4    12           44
13             4    13           51
14             4    14           48
15             4    15           66
16             4    16           84
17             4    17          100
18             4    18          135
19             4    19          176
20             4    20          122
21             4    21          140
22             4    22          125
23             4    23          136

Category: 12
    DOLocationID  time  num_pickups
0             12     0            3
19

In [199]:
df_ThreeKings= pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_ThreeKings= pd.concat([df_ThreeKings,df],axis=0)
df_ThreeKings

,DOLocationID,time,num_pickups
0,4,0,28
1,4,1,18
2,4,2,11
3,4,3,11
4,4,4,10
...,...,...,...
1501,263,19,505
1502,263,20,430
1503,263,21,328
1504,263,22,388


## baseline

In [200]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-01.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-01-12")]
df

,DOLocationID,date,month,time,day_of_the_week
2587374,161,2018-01-12,1,6,Friday
2587375,162,2018-01-12,1,6,Friday
2587376,162,2018-01-12,1,6,Friday
2587377,229,2018-01-12,1,6,Friday
2602401,142,2018-01-12,1,6,Friday
...,...,...,...,...,...
3148432,74,2018-01-12,1,23,Friday
3150382,186,2018-01-12,1,21,Friday
3410618,186,2018-01-12,1,23,Friday
3419240,234,2018-01-12,1,23,Friday


In [201]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0           74
1              4     1           42
2              4     2           32
3              4     3           24
4              4     4           15
5              4     5           11
6              4     6           12
7              4     7           38
8              4     8           38
9              4     9           14
10             4    10           28
11             4    11           39
12             4    12           45
13             4    13           47
14             4    14           71
15             4    15           80
16             4    16           67
17             4    17          106
18             4    18          131
19             4    19          203
20             4    20          146
21             4    21          129
22             4    22          116
23             4    23          168

Category: 12
    DOLocationID  time  num_pickups
24            12     0            2
25

In [202]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
dfs        

{4:     DOLocationID  time  num_pickups
 0              4     0           74
 1              4     1           42
 2              4     2           32
 3              4     3           24
 4              4     4           15
 5              4     5           11
 6              4     6           12
 7              4     7           38
 8              4     8           38
 9              4     9           14
 10             4    10           28
 11             4    11           39
 12             4    12           45
 13             4    13           47
 14             4    14           71
 15             4    15           80
 16             4    16           67
 17             4    17          106
 18             4    18          131
 19             4    19          203
 20             4    20          146
 21             4    21          129
 22             4    22          116
 23             4    23          168,
 12:     DOLocationID  time  num_pickups
 0             12     0       

In [203]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
# df_bl= pd.DataFrame(df_bl)
df_bl

,DOLocationID,time,num_pickups
0,4,0,74
1,4,1,42
2,4,2,32
3,4,3,24
4,4,4,15
...,...,...,...
1525,263,19,544
1526,263,20,452
1527,263,21,414
1528,263,22,355


In [204]:
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl

,DOLocationID,time,num_pickups
0,4,0,74
1,4,1,42
2,4,2,32
3,4,3,24
4,4,4,15
...,...,...,...
1579,263,19,544
1580,263,20,452
1581,263,21,414
1582,263,22,355


In [205]:
df_ThreeKings.reset_index(drop=True, inplace=True)
df_ThreeKings = df_ThreeKings.rename(columns={'num_pickups':'num_pickups_ThreeKings'})
df_ThreeKings

,DOLocationID,time,num_pickups_ThreeKings
0,4,0,28
1,4,1,18
2,4,2,11
3,4,3,11
4,4,4,10
...,...,...,...
1579,263,19,505
1580,263,20,430
1581,263,21,328
1582,263,22,388


In [206]:
df_compare= pd.concat([df_bl,df_ThreeKings["num_pickups_ThreeKings"]],axis=1)
df_compare

,DOLocationID,time,num_pickups,num_pickups_ThreeKings
0,4,0,74,28
1,4,1,42,18
2,4,2,32,11
3,4,3,24,11
4,4,4,15,10
...,...,...,...,...
1579,263,19,544,505
1580,263,20,452,430
1581,263,21,414,328
1582,263,22,355,388


In [207]:
average = df_compare['num_pickups'].mean()
average_TS = df_compare['num_pickups_ThreeKings'].mean()
print(average)
print(average_TS)

178.01767676767676
148.85416666666666


## models and predictions

In [209]:
X = df_compare.drop('num_pickups_ThreeKings', axis=1)
y = df_compare['num_pickups_ThreeKings']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [210]:
xg_reg = XGBRegressor(objective ='reg:squarederror', eval_metric ='rmse')
xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

In [211]:
print("\n==================== Test Data =======================")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, preds))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, preds))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('R^2:', metrics.r2_score(y_test, preds))
print("\n=======================================================")


==================== Test Data =======================
Mean Absolute Error: 17.43081899006076
Mean Squared Error: 859.0452104721928
Root Mean Squared Error: 29.30947305005999
R^2: 0.9703071068176761



In [212]:
preds = xg_reg.predict(X_test)
true_vs_predicted_test = pd.DataFrame({'Actual Value': y_test, 'Predicted Value': preds})
# true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
# true_vs_predicted_test = pd.concat([X_test,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,Actual Value,Predicted Value
944,502,396.728180
1032,60,59.007290
893,180,186.070618
667,272,364.027527
1256,43,45.604141
...,...,...
1365,291,358.528839
842,0,0.164014
1199,268,306.818542
790,381,348.148621


In [213]:
df_2022 = pd.read_json('predictions.json')
df_2022 = df_2022[(df_2022['month'] == 1) & (df_2022['day_of_the_week'] == 4)]
df_2022.reset_index(drop=True, inplace=True)
df_2022=df_2022.drop(["month","day_of_the_week","busyness_score"], axis =1)
df_2022=df_2022.rename(columns={'ZoneID':'DOLocationID','predicted_dropoffs':'num_pickups'})
df_2022

,DOLocationID,time,num_pickups
0,236,17,1299.138306
1,236,21,721.990051
2,236,11,1166.497559
3,236,22,614.060303
4,236,9,891.017761
...,...,...,...
1603,105,23,581.604248
1604,105,16,451.948761
1605,105,8,245.106964
1606,105,3,82.377510


In [214]:
preds = xg_reg.predict(df_2022)
true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
true_vs_predicted_test = pd.concat([df_2022,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
0,236,17,1299.138306,1076.763794
1,236,21,721.990051,590.492554
2,236,11,1166.497559,895.123047
3,236,22,614.060303,490.715210
4,236,9,891.017761,604.778992
...,...,...,...,...
1603,105,23,581.604248,506.764069
1604,105,16,451.948761,426.796448
1605,105,8,245.106964,187.408936
1606,105,3,82.377510,37.468086


In [215]:
true_vs_predicted_test = true_vs_predicted_test.sort_values(by=['DOLocationID', 'time'])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
610,4,0,45.170708,16.559225
618,4,1,63.532543,31.715433
615,4,2,29.534712,12.297634
622,4,3,9.751890,6.168545
612,4,4,-16.819645,0.521829
...,...,...,...,...
369,263,19,608.155762,557.657776
374,263,20,500.246582,424.167603
361,263,21,452.304199,366.613800
363,263,22,504.579742,364.427948


In [216]:
# Normalize predictions to [0,1] range
scaler = MinMaxScaler()
true_vs_predicted_test['busyness_score'] = scaler.fit_transform(true_vs_predicted_test[['Predicted Value']])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score
610,4,0,45.170708,16.559225,0.019257
618,4,1,63.532543,31.715433,0.032946
615,4,2,29.534712,12.297634,0.015409
622,4,3,9.751890,6.168545,0.009873
612,4,4,-16.819645,0.521829,0.004773
...,...,...,...,...,...
369,263,19,608.155762,557.657776,0.507952
374,263,20,500.246582,424.167603,0.387390
361,263,21,452.304199,366.613800,0.335410
363,263,22,504.579742,364.427948,0.333436


In [217]:
true_vs_predicted_test["Event_ID"]=7
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score,Event_ID
610,4,0,45.170708,16.559225,0.019257,7
618,4,1,63.532543,31.715433,0.032946,7
615,4,2,29.534712,12.297634,0.015409,7
622,4,3,9.751890,6.168545,0.009873,7
612,4,4,-16.819645,0.521829,0.004773,7
...,...,...,...,...,...,...
369,263,19,608.155762,557.657776,0.507952,7
374,263,20,500.246582,424.167603,0.387390,7
361,263,21,452.304199,366.613800,0.335410,7
363,263,22,504.579742,364.427948,0.333436,7


In [218]:
true_vs_predicted_test = true_vs_predicted_test.rename(columns={'Predicted Value':'predicted_value'})
true_vs_predicted_test=true_vs_predicted_test[["Event_ID","DOLocationID","time","num_pickups","predicted_value","busyness_score"]]
true_vs_predicted_test.reset_index(drop=True, inplace=True)
true_vs_predicted_test

,Event_ID,DOLocationID,time,num_pickups,predicted_value,busyness_score
0,7,4,0,45.170708,16.559225,0.019257
1,7,4,1,63.532543,31.715433,0.032946
2,7,4,2,29.534712,12.297634,0.015409
3,7,4,3,9.751890,6.168545,0.009873
4,7,4,4,-16.819645,0.521829,0.004773
...,...,...,...,...,...,...
1603,7,263,19,608.155762,557.657776,0.507952
1604,7,263,20,500.246582,424.167603,0.387390
1605,7,263,21,452.304199,366.613800,0.335410
1606,7,263,22,504.579742,364.427948,0.333436


In [219]:
true_vs_predicted_test.to_json('ThreeKings.json', orient='records')
with open('ThreeKings.pkl', 'wb') as file:
    pickle.dump(xg_reg, file)

# Macy 4th of July

## actual data

In [220]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-07.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-07-04")]
df

,DOLocationID,date,month,time,day_of_the_week
714412,163,2018-07-04,7,0,Wednesday
714461,113,2018-07-04,7,0,Wednesday
714789,163,2018-07-04,7,0,Wednesday
715120,13,2018-07-04,7,0,Wednesday
715311,141,2018-07-04,7,0,Wednesday
...,...,...,...,...,...
7850613,79,2018-07-04,7,0,Wednesday
7850615,262,2018-07-04,7,12,Wednesday
7850616,74,2018-07-04,7,13,Wednesday
7850618,233,2018-07-04,7,18,Wednesday


In [221]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0           76
1              4     1           53
2              4     2           41
3              4     3           35
4              4     4           25
5              4     5            7
6              4     6            8
7              4     7           14
8              4     8           11
9              4     9           18
10             4    10           21
11             4    11           18
12             4    12           43
13             4    13           49
14             4    14           48
15             4    15           49
16             4    16           71
17             4    17           49
18             4    18          103
19             4    19           95
20             4    20          111
21             4    21          112
22             4    22           46
23             4    23           82

Category: 12
    DOLocationID  time  num_pickups
24            12     0            2
25

In [222]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
        
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0           76
1              4     1           53
2              4     2           41
3              4     3           35
4              4     4           25
5              4     5            7
6              4     6            8
7              4     7           14
8              4     8           11
9              4     9           18
10             4    10           21
11             4    11           18
12             4    12           43
13             4    13           49
14             4    14           48
15             4    15           49
16             4    16           71
17             4    17           49
18             4    18          103
19             4    19           95
20             4    20          111
21             4    21          112
22             4    22           46
23             4    23           82

Category: 12
    DOLocationID  time  num_pickups
0             12     0            2
1 

      DOLocationID  time  num_pickups
1476           262     0           86
1477           262     1           41
1478           262     2           39
1479           262     3           22
1480           262     4           15
1481           262     5           19
1482           262     6           14
1483           262     7           23
1484           262     8           26
1485           262     9           55
1486           262    10           62
1487           262    11           89
1488           262    12           95
1489           262    13           98
1490           262    14          105
1491           262    15           91
1492           262    16          113
1493           262    17          106
1494           262    18          135
1495           262    19          124
1496           262    20          159
1497           262    21          156
1498           262    22          173
1499           262    23          172

Category: 263
      DOLocationID  time  num_picku

In [223]:
df_Macy= pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_Macy= pd.concat([df_Macy,df],axis=0)
df_Macy

,DOLocationID,time,num_pickups
0,4,0,76
1,4,1,53
2,4,2,41
3,4,3,35
4,4,4,25
...,...,...,...
1519,263,19,229
1520,263,20,187
1521,263,21,165
1522,263,22,306


## baseline

In [225]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-07.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-07-11")]
df

,DOLocationID,date,month,time,day_of_the_week
1998665,79,2018-07-11,7,3,Wednesday
2001285,88,2018-07-11,7,5,Wednesday
2008472,13,2018-07-11,7,5,Wednesday
2008473,163,2018-07-11,7,6,Wednesday
2008474,163,2018-07-11,7,6,Wednesday
...,...,...,...,...,...
2566621,107,2018-07-11,7,18,Wednesday
2571712,234,2018-07-11,7,12,Wednesday
2834093,141,2018-07-11,7,22,Wednesday
7850652,100,2018-07-11,7,22,Wednesday


In [226]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_pickups')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

# Access and print the separate DataFrames
for category, df in dfs.items():
    print(f"Category: {category}")
    print(df)
    print()

Category: 4
    DOLocationID  time  num_pickups
0              4     0           51
1              4     1           28
2              4     2           23
3              4     3           13
4              4     4            4
5              4     5            5
6              4     6           14
7              4     7           21
8              4     8           15
9              4     9           20
10             4    10           28
11             4    11           23
12             4    12           39
13             4    13           47
14             4    14           38
15             4    15           47
16             4    16           60
17             4    17           79
18             4    18          113
19             4    19           96
20             4    20           98
21             4    21           99
22             4    22           95
23             4    23          118

Category: 12
    DOLocationID  time  num_pickups
24            12     0            8
25

In [227]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_pickups":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df
dfs        

{4:     DOLocationID  time  num_pickups
 0              4     0           51
 1              4     1           28
 2              4     2           23
 3              4     3           13
 4              4     4            4
 5              4     5            5
 6              4     6           14
 7              4     7           21
 8              4     8           15
 9              4     9           20
 10             4    10           28
 11             4    11           23
 12             4    12           39
 13             4    13           47
 14             4    14           38
 15             4    15           47
 16             4    16           60
 17             4    17           79
 18             4    18          113
 19             4    19           96
 20             4    20           98
 21             4    21           99
 22             4    22           95
 23             4    23          118,
 12:     DOLocationID  time  num_pickups
 0             12     0       

In [228]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
#     print(f"Category: {category}")
#     print(df)
#     print()
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
# df_bl= pd.DataFrame(df_bl)
df_bl

,DOLocationID,time,num_pickups
0,4,0,51
1,4,1,28
2,4,2,23
3,4,3,13
4,4,4,4
...,...,...,...
1522,263,19,483
1523,263,20,446
1524,263,21,425
1525,263,22,379


In [229]:
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl

,DOLocationID,time,num_pickups
0,4,0,51
1,4,1,28
2,4,2,23
3,4,3,13
4,4,4,4
...,...,...,...
1579,263,19,483
1580,263,20,446
1581,263,21,425
1582,263,22,379


In [230]:
df_Macy.reset_index(drop=True, inplace=True)
df_Macy = df_Macy.rename(columns={'num_pickups':'num_pickups_Macy'})
df_Macy

,DOLocationID,time,num_pickups_Macy
0,4,0,76
1,4,1,53
2,4,2,41
3,4,3,35
4,4,4,25
...,...,...,...
1579,263,19,229
1580,263,20,187
1581,263,21,165
1582,263,22,306


In [231]:
df_compare= pd.concat([df_bl,df_Macy["num_pickups_Macy"]],axis=1)
df_compare

,DOLocationID,time,num_pickups,num_pickups_Macy
0,4,0,51,76
1,4,1,28,53
2,4,2,23,41
3,4,3,13,35
4,4,4,4,25
...,...,...,...,...
1579,263,19,483,229
1580,263,20,446,187
1581,263,21,425,165
1582,263,22,379,306


In [232]:
average = df_compare['num_pickups'].mean()
average_TS = df_compare['num_pickups_Macy'].mean()
print(average)
print(average_TS)

161.90088383838383
96.62563131313131


## models and predictions

In [237]:
X = df_compare.drop('num_pickups_Macy', axis=1)
y = df_compare['num_pickups_Macy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [238]:
xg_reg = XGBRegressor(objective ='reg:squarederror', eval_metric ='rmse')
xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

In [239]:
print("\n==================== Test Data =======================")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, preds))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, preds))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, preds)))
print('R^2:', metrics.r2_score(y_test, preds))
print("\n=======================================================")


==================== Test Data =======================
Mean Absolute Error: 23.091117023936466
Mean Squared Error: 1729.9029378733494
Root Mean Squared Error: 41.59210186890474
R^2: 0.8225285568956049



In [240]:
preds = xg_reg.predict(X_test)
true_vs_predicted_test = pd.DataFrame({'Actual Value': y_test, 'Predicted Value': preds})
# true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
# true_vs_predicted_test = pd.concat([X_test,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,Actual Value,Predicted Value
944,90,70.255615
1032,226,218.082031
893,47,45.256985
667,166,151.298462
1256,12,11.508785
...,...,...
1365,156,217.966309
842,1,0.224122
1199,202,297.038025
790,129,171.957001


In [241]:
df_2022 = pd.read_json('predictions.json')
df_2022 = df_2022[(df_2022['month'] == 7) & (df_2022['day_of_the_week'] == 2)]
df_2022.reset_index(drop=True, inplace=True)
df_2022=df_2022.drop(["month","day_of_the_week","busyness_score"], axis =1)
df_2022=df_2022.rename(columns={'ZoneID':'DOLocationID','predicted_dropoffs':'num_pickups'})
df_2022

,DOLocationID,time,num_pickups
0,236,17,1272.928833
1,236,21,1058.624023
2,236,11,1109.233032
3,236,22,758.606934
4,236,9,851.047058
...,...,...,...
1603,105,23,377.441620
1604,105,16,460.230835
1605,105,8,282.843262
1606,105,3,30.947538


In [242]:
preds = xg_reg.predict(df_2022)
true_vs_predicted_test = pd.DataFrame({'Predicted Value': preds})
true_vs_predicted_test = pd.concat([df_2022,true_vs_predicted_test],axis=1)
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
0,236,17,1272.928833,258.885620
1,236,21,1058.624023,274.163208
2,236,11,1109.233032,281.578064
3,236,22,758.606934,305.851898
4,236,9,851.047058,141.524506
...,...,...,...,...
1603,105,23,377.441620,296.837860
1604,105,16,460.230835,430.426422
1605,105,8,282.843262,81.582970
1606,105,3,30.947538,58.865677


In [243]:
true_vs_predicted_test = true_vs_predicted_test.sort_values(by=['DOLocationID', 'time'])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value
610,4,0,51.908276,75.243690
618,4,1,58.406055,121.752342
615,4,2,40.327198,97.496185
622,4,3,33.016911,60.180435
612,4,4,3.597897,15.123820
...,...,...,...,...
369,263,19,707.040466,234.178619
374,263,20,640.171143,236.355057
361,263,21,654.346741,276.218811
363,263,22,636.307556,318.504242


In [244]:
# Normalize predictions to [0,1] range
scaler = MinMaxScaler()
true_vs_predicted_test['busyness_score'] = scaler.fit_transform(true_vs_predicted_test[['Predicted Value']])
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score
610,4,0,51.908276,75.243690,0.160878
618,4,1,58.406055,121.752342,0.240492
615,4,2,40.327198,97.496185,0.198970
622,4,3,33.016911,60.180435,0.135092
612,4,4,3.597897,15.123820,0.057963
...,...,...,...,...,...
369,263,19,707.040466,234.178619,0.432946
374,263,20,640.171143,236.355057,0.436672
361,263,21,654.346741,276.218811,0.504911
363,263,22,636.307556,318.504242,0.577297


In [245]:
true_vs_predicted_test["Event_ID"]=8
true_vs_predicted_test

,DOLocationID,time,num_pickups,Predicted Value,busyness_score,Event_ID
610,4,0,51.908276,75.243690,0.160878,8
618,4,1,58.406055,121.752342,0.240492,8
615,4,2,40.327198,97.496185,0.198970,8
622,4,3,33.016911,60.180435,0.135092,8
612,4,4,3.597897,15.123820,0.057963,8
...,...,...,...,...,...,...
369,263,19,707.040466,234.178619,0.432946,8
374,263,20,640.171143,236.355057,0.436672,8
361,263,21,654.346741,276.218811,0.504911,8
363,263,22,636.307556,318.504242,0.577297,8


In [246]:
true_vs_predicted_test = true_vs_predicted_test.rename(columns={'Predicted Value':'predicted_value'})
true_vs_predicted_test=true_vs_predicted_test[["Event_ID","DOLocationID","time","num_pickups","predicted_value","busyness_score"]]
true_vs_predicted_test.reset_index(drop=True, inplace=True)
true_vs_predicted_test

,Event_ID,DOLocationID,time,num_pickups,predicted_value,busyness_score
0,8,4,0,51.908276,75.243690,0.160878
1,8,4,1,58.406055,121.752342,0.240492
2,8,4,2,40.327198,97.496185,0.198970
3,8,4,3,33.016911,60.180435,0.135092
4,8,4,4,3.597897,15.123820,0.057963
...,...,...,...,...,...,...
1603,8,263,19,707.040466,234.178619,0.432946
1604,8,263,20,640.171143,236.355057,0.436672
1605,8,263,21,654.346741,276.218811,0.504911
1606,8,263,22,636.307556,318.504242,0.577297


In [247]:
true_vs_predicted_test.to_json('Macy4thJuly.json', orient='records')
with open('Macy4thJuly.pkl', 'wb') as file:
    pickle.dump(xg_reg, file)